# 2_Var_LCA

## Project: LCA - Low Carbon Agriculture India

##  This Python script does the following


    * uses clean data from first stage cleaning 
    
    * constructs variable for selected indicators for LCA 
    
    * saves data with key indicators as HH_lca_FINAL_Contsruction



In [1]:
# Importing Required Libraries

import pandas as pd
import numpy as np

# Plotting Libraries

import seaborn as sns
import matplotlib.pyplot as plt


# PCA
from sklearn.decomposition import PCA
from statsmodels.stats.correlation_tools import corr_nearest

import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')




In [2]:


# Specify the path to your .dta file
df = pd.read_csv('/Users/la/Downloads/LCA_Endline_Final_Clean_4(1).dta')


# Display the DataFrame
df.head()



,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,total_farm_size,exceeds_farm_size,crop_unit_name_1,crop_unit_name_2,crop_unit_name_3,crop_unit_name_4,crop_unit_name_5,res_gender_new,fies_raw,fies_latent
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,44.0,False,NaN,NaN,Man,NaN,NaN,NaN,0.0,0.196821
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,21.0,False,NaN,NaN,Man,NaN,NaN,NaN,0.0,0.196821
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,6.0,False,NaN,NaN,Man,NaN,NaN,Male,2.0,0.196821
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,8.0,False,NaN,NaN,Man,NaN,NaN,NaN,0.0,0.196821
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,18.5,False,NaN,NaN,Man,NaN,NaN,Male,0.0,0.196821


## Major Crops grown in Kaharif 2023

## Drop crop varities for which there is 0% prevalence.

In [3]:
# Define your crop variables with comprehensive list comprehension
crop_vars = [f"crop_var_23_{i}_{j}" for i in range(1, 45) for j in range(1, 7)]

# Calculate means and remove columns with mean of 0
for var in crop_vars:
    if var in df.columns:
        mean_val = df[var].mean()
        if mean_val == 0:
            df.drop(columns=[var], inplace=True)

# Setting specific variables to zero
for i in [1, 4, 5, 6]:
    df[f'crop_var_23_43_{i}'] = 0
    if i in [1, 2, 5, 6]:
        df[f'crop_var_23_10_{i}'] = 0


## Number of plots for each crop 	 

In [4]:
# List of indices for the crop variables as specified in the STATA command
indices = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 32, 33, 40, 42, 43, 44]

# Generate total plots for each specified index
for i in indices:
    # Create a list of all columns that match the pattern for this index
    columns = [col for col in df.columns if col.startswith(f'crop_var_23_{i}_')]
    # Calculate the row total for the identified columns
    df[f'tot_plot_{i}'] = df[columns].sum(axis=1)

# Crops Grown by Households

In [5]:


# Define a dictionary for crop list mapping
crop_list = {
    1: "Paddy rice", 2: "Cotton", 3: "Chillies", 4: "Maize", 5: "Redgram", 6: "Greengram",
    10: "Turmeric", 11: "Groundnut", 12: "Soybean", 13: "Jowar (Sorghum)", 14: "Bajra (Pearl Millet)",
    17: "Castor", 19: "Sessamun", 20: "Sugarcane", 21: "Wheat", 22: "Mustard", 23: "Banana",
    24: "Papaya", 25: "Mango", 26: "Citrus", 28: "Sapota", 32: "Guava", 33: "Coconut",
    40: "Flowers", 42: "Timber trees", 43: "Fodder & Shrubs", 44: "Other"
}

indices = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 32, 33, 40, 42, 43, 44]
for i in indices:
    # Creating the column names list for tot_plot calculation
    columns = [col for col in df.columns if col.startswith(f'crop_var_23_{i}_')]
    
    # Check if there are any columns to sum up, then create tot_plot
    if columns:
        df[f'tot_plot_{i}'] = df[columns].sum(axis=1)
    
    # Create the crop_varty column and set to 0, update based on conditions only if tot_plot column exists
    crop_varty_col = f'crop_varty_{i}'
    df[crop_varty_col] = 0
    if f'tot_plot_{i}' in df.columns:
        df.loc[df[f'tot_plot_{i}'] > 0, crop_varty_col] = 1




In [6]:
# Checking if the specific columns tot_plot_1 exists

print(df.columns)
indices = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 32, 33, 40, 42, 43, 44]
for i in indices:
    print(f"tot_plot_{i} exists in DataFrame:", f"tot_plot_{i}" in df.columns)
print(df.info())

# checking if they are correctly populated by checking the count
print(df[[f"tot_plot_{i}" for i in indices if f"tot_plot_{i}" in df.columns]].count())



Index(['deviceid', 'device_num', 'text_audit', 'state_name', 'dis_name',
       'block', 'village_name', 'audio_audit_1', 'audio_audit_2',
       'audio_audit_3',
       ...
       'crop_varty_24', 'crop_varty_25', 'crop_varty_26', 'crop_varty_28',
       'crop_varty_32', 'crop_varty_33', 'crop_varty_40', 'crop_varty_42',
       'crop_varty_43', 'crop_varty_44'],
      dtype='object', length=11501)
tot_plot_1 exists in DataFrame: True
tot_plot_2 exists in DataFrame: True
tot_plot_3 exists in DataFrame: True
tot_plot_4 exists in DataFrame: True
tot_plot_5 exists in DataFrame: True
tot_plot_6 exists in DataFrame: True
tot_plot_10 exists in DataFrame: True
tot_plot_11 exists in DataFrame: True
tot_plot_12 exists in DataFrame: True
tot_plot_13 exists in DataFrame: True
tot_plot_14 exists in DataFrame: True
tot_plot_17 exists in DataFrame: True
tot_plot_19 exists in DataFrame: True
tot_plot_20 exists in DataFrame: True
tot_plot_21 exists in DataFrame: True
tot_plot_22 exists in DataFrame: T

## Share of total cultivated area

In [7]:


# Define crop list mapping (update with actual crop names)
crop_list = {
    1: "Crop 1", 6: "Crop 6", 10: "Crop 10", 14: "Crop 14", 17: "Crop 17", 19: "Crop 19",
    26: "Crop 26", 28: "Crop 28", 32: "Crop 32", 33: "Crop 33", 40: "Crop 40", 42: "Crop 42", 43: "Crop 43"
}

indices = [1, 6, 10, 14, 17, 19, 26, 28, 32, 33, 40, 42, 43]
for i in indices:
    df[f'crop_share_{i}'] = df[f'farm_size_crp_tot_{i}'] / df['f_size_total'] * 100
    df[f'crop_share_per_{i}'] = df[f'crop_share_{i}']


In [8]:
# Summarize crop_area_tot and apply outlier adjustments
crop_area_tot_details = df['crop_area_tot'].describe()
q75, q25 = crop_area_tot_details['75%'], crop_area_tot_details['25%']
iqr = q75 - q25
upper_bound = q75 + (3 * iqr)
lower_bound = q25 - (3 * iqr)

df['crop_area_tot'] = df['crop_area_tot'].clip(lower=lower_bound, upper=upper_bound)


In [9]:
for i in indices:
    # Replace missing values with zero
    df[f'crop_varty_{i}'].fillna(0, inplace=True)
    
    # Convert to percentage
    df[f'crop_varty_per_{i}'] = df[f'crop_varty_{i}'] * 100


In [10]:
# confirming that the columns are present

print(df.info())
indices = [1, 6, 10, 14, 17, 19, 26, 28, 32, 33, 40, 42, 43]
for i in indices:
    for col_suffix in ['crop_share_', 'crop_share_per_', 'crop_varty_per_']:
        col_name = f'{col_suffix}{i}'
        if col_name in df.columns:
            print(f"{col_name} found, type: {df[col_name].dtype}")
        else:
            print(f"{col_name} not found in DataFrame")

for i in indices:
    for col_suffix in ['crop_share_', 'crop_share_per_', 'crop_varty_per_']:
        col_name = f'{col_suffix}{i}'
        if col_name in df.columns:
            print(f"Statistics for {col_name}:")
            print(df[col_name].describe())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1046 entries, 0 to 1045
Columns: 11540 entries, deviceid to crop_varty_per_43
dtypes: bool(1), float64(10571), int64(191), object(777)
memory usage: 92.1+ MB
None
crop_share_1 found, type: float64
crop_share_per_1 found, type: float64
crop_varty_per_1 found, type: int64
crop_share_6 found, type: float64
crop_share_per_6 found, type: float64
crop_varty_per_6 found, type: int64
crop_share_10 found, type: float64
crop_share_per_10 found, type: float64
crop_varty_per_10 found, type: int64
crop_share_14 found, type: float64
crop_share_per_14 found, type: float64
crop_varty_per_14 found, type: int64
crop_share_17 found, type: float64
crop_share_per_17 found, type: float64
crop_varty_per_17 found, type: int64
crop_share_19 found, type: float64
crop_share_per_19 found, type: float64
crop_varty_per_19 found, type: int64
crop_share_26 found, type: float64
crop_share_per_26 found, type: float64
crop_varty_per_26 found, type: int64
crop_share_28 fo

# Major crops grown  kaharif 2024

##  Drop crop varities for which there is 0% prevalence.

In [11]:
# Defining the variable list
variable_names = """
crop_var_24_1_1 crop_var_24_2_1 crop_var_24_3_1 _24 crop_var_24_5_1 crop_var_24_6_1 crop_var_24_7_1 crop_var_24_8_1 crop_var_24_9_1 crop_var_24_10_1 crop_var_24_11_1 crop_var_24_12_1 crop_var_24_13_1 crop_var_24_14_1 crop_var_24_15_1 crop_var_24_16_1 crop_var_24_17_1 crop_var_24_18_1 crop_var_24_19_1 crop_var_24_24_1 crop_var_24_23_1 crop_var_24_22_1 crop_var_24_24_1 crop_var_24_24_1 crop_var_24_25_1 crop_var_24_26_1 crop_var_24_27_1 crop_var_24_28_1 crop_var_24_29_1 crop_var_24_30_1 crop_var_24_31_1 crop_var_24_32_1 crop_var_24_33_1 crop_var_24_34_1 crop_var_24_35_1 crop_var_24_36_1 crop_var_24_37_1 crop_var_24_38_1 crop_var_24_39_1 crop_var_24_40_1 crop_var_24_41_1 crop_var_24_42_1 crop_var_24_43_1 crop_var_24_44_1 crop_var_24_1_2 crop_var_24_2_2 crop_var_24_3_2 crop_var_24_4_2 crop_var_24_5_2 crop_var_24_6_2 crop_var_24_7_2 crop_var_24_8_2 crop_var_24_9_2 crop_var_24_10_2 crop_var_24_11_2 crop_var_24_12_2 crop_var_24_13_2 crop_var_24_14_2 crop_var_24_15_2 crop_var_24_16_2 crop_var_24_17_2 crop_var_24_18_2 crop_var_24_19_2 crop_var_24_24_2 crop_var_24_23_2 crop_var_24_22_2 crop_var_24_24_2 crop_var_24_24_2 crop_var_24_25_2 crop_var_24_26_2 crop_var_24_27_2 crop_var_24_28_2 crop_var_24_29_2 crop_var_24_30_2 crop_var_24_31_2 crop_var_24_32_2 crop_var_24_33_2 crop_var_24_34_2 crop_var_24_35_2 crop_var_24_36_2 crop_var_24_37_2 crop_var_24_38_2 crop_var_24_39_2 crop_var_24_40_2 crop_var_24_41_2 crop_var_24_42_2 crop_var_24_43_2 crop_var_24_44_2 crop_var_24_1_3 crop_var_24_2_3 crop_var_24_3_3 crop_var_24_4_3 crop_var_24_5_3 crop_var_24_6_3 crop_var_24_7_3 crop_var_24_8_3 crop_var_24_9_3 crop_var_24_10_3 crop_var_24_11_3 crop_var_24_12_3 crop_var_24_13_3 crop_var_24_14_3 crop_var_24_15_3 crop_var_24_16_3 crop_var_24_17_3 crop_var_24_18_3 crop_var_24_19_3 crop_var_24_24_3 crop_var_24_23_3 crop_var_24_22_3 crop_var_24_24_3 crop_var_24_24_3 crop_var_24_25_3 crop_var_24_26_3 crop_var_24_27_3 crop_var_24_28_3 crop_var_24_29_3 crop_var_24_30_3 crop_var_24_31_3 crop_var_24_32_3 crop_var_24_33_3 crop_var_24_34_3 crop_var_24_35_3 crop_var_24_36_3 crop_var_24_37_3 crop_var_24_38_3 crop_var_24_39_3 crop_var_24_40_3 crop_var_24_41_3 crop_var_24_42_3 crop_var_24_43_3 crop_var_24_44_3 crop_var_24_1_4 crop_var_24_2_4 crop_var_24_3_4 crop_var_24_4_4 crop_var_24_5_4 crop_var_24_6_4 crop_var_24_7_4 crop_var_24_8_4 crop_var_24_9_4 crop_var_24_10_4 crop_var_24_11_4 crop_var_24_12_4 crop_var_24_13_4 crop_var_24_14_4 crop_var_24_15_4 crop_var_24_16_4 crop_var_24_17_4 crop_var_24_18_4 crop_var_24_19_4 crop_var_24_24_4 crop_var_24_23_4 crop_var_24_22_4 crop_var_24_24_4 crop_var_24_24_4 crop_var_24_25_4 crop_var_24_26_4 crop_var_24_27_4 crop_var_24_28_4 crop_var_24_29_4 crop_var_24_30_4 crop_var_24_31_4 crop_var_24_32_4 crop_var_24_33_4 crop_var_24_34_4 crop_var_24_35_4 crop_var_24_36_4 crop_var_24_37_4 crop_var_24_38_4 crop_var_24_39_4 crop_var_24_40_4 crop_var_24_41_4 crop_var_24_42_4 crop_var_24_43_4 crop_var_24_44_4 crop_var_24_1_5 crop_var_24_2_5 crop_var_24_3_5 crop_var_24_4_5 crop_var_24_5_5 crop_var_24_6_5 crop_var_24_7_5 crop_var_24_8_5 crop_var_24_9_5 crop_var_24_10_5 crop_var_24_11_5 crop_var_24_12_5 crop_var_24_13_5 crop_var_24_14_5 crop_var_24_15_5 crop_var_24_16_5 crop_var_24_17_5 crop_var_24_18_5 crop_var_24_19_5 crop_var_24_24_5 crop_var_24_23_5 crop_var_24_22_5 crop_var_24_24_5 crop_var_24_24_5 crop_var_24_25_5 crop_var_24_26_5 crop_var_24_27_5 crop_var_24_28_5 crop_var_24_29_5 crop_var_24_30_5 crop_var_24_31_5 crop_var_24_32_5 crop_var_24_33_5 crop_var_24_34_5 crop_var_24_35_5 crop_var_24_36_5 crop_var_24_37_5 crop_var_24_38_5 crop_var_24_39_5 crop_var_24_40_5 crop_var_24_41_5 crop_var_24_42_5 crop_var_24_43_5 crop_var_24_44_5 crop_var_24_1_6 crop_var_24_2_6 crop_var_24_3_6 crop_var_24_4_6 crop_var_24_5_6 crop_var_24_6_6 crop_var_24_7_6 crop_var_24_8_6 crop_var_24_9_6 crop_var_24_10_6 crop_var_24_11_6 crop_var_24_12_6 crop_var_24_13_6 crop_var_24_14_6 crop_var_24_15_6 crop_var_24_16_6 crop_var_24_17_6 crop_var_24_18_6 crop_var_24_19_6 crop_var_24_24_6 crop_var_24_23_6 crop_var_24_22_6 crop_var_24_24_6 crop_var_24_24_6 crop_var_24_25_6 crop_var_24_26_6 crop_var_24_27_6 crop_var_24_28_6 crop_var_24_29_6 crop_var_24_30_6 crop_var_24_31_6 crop_var_24_32_6 crop_var_24_33_6 crop_var_24_34_6 crop_var_24_35_6 crop_var_24_36_6 crop_var_24_37_6 crop_var_24_38_6 crop_var_24_39_6 crop_var_24_40_6 crop_var_24_41_6 crop_var_24_42_6 crop_var_24_43_6 crop_var_24_44_6"
"""

# Create a dictionary using list comprehension
variable_dict = {var: 1 for var in variable_names.split()}




In [12]:

# Process each variable in the dictionary to calculate the mean and replace values
for var in variable_dict.keys():
    if var in df.columns:
        # Calculate mean
        mean_val = df[var].mean()

        # Replace the column values with NaN if the mean is zero
        if mean_val == 0:
            df[var] = np.nan

        # Drop the column if all values are now NaN
        if df[var].isna().all():
            df.drop(columns=[var], inplace=True)

        # Drop the mean column if used (not necessary in pandas but mimicking STATA logic)
        if f'{var}_mean' in df.columns:
            df.drop(columns=[f'{var}_mean'], inplace=True)


In [13]:
# Set specific crop variables to zero
indices_17 = [1, 3, 4, 5, 6]
for i in indices_17:
    col_name = f'crop_var_24_17_{i}'
    if col_name in df.columns:
        df[col_name] = 0

indices_10 = [1, 2, 5, 6]
for i in indices_10:
    col_name = f'crop_var_24_10_{i}'
    if col_name in df.columns:
        df[col_name] = 0

indices_43 = [1, 4, 5, 6]
for i in indices_43:
    col_name = f'crop_var_24_43_{i}'
    if col_name in df.columns:
        df[col_name] = 0


## Number of plots for each crop

In [14]:


# Define the indices for which you need to calculate the total plots
indices = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 32, 33, 40, 42, 43, 44]

for i in indices:
    # Generate column names based on the pattern
    columns = [col for col in df.columns if col.startswith(f'crop_var_24_{i}_')]
    
    # Calculate the row total of these columns
    df[f'tot_plot_24_{i}'] = df[columns].sum(axis=1)


##  crops grown by households

In [15]:


# Define the crop list mapping (update with actual crop names)
crop_list = {
    1: "Crop 1", 6: "Crop 6", 10: "Crop 10", 14: "Crop 14", 17: "Crop 17", 19: "Crop 19",
    26: "Crop 26", 28: "Crop 28", 32: "Crop 32", 33: "Crop 33", 40: "Crop 40", 42: "Crop 42",
    43: "Crop 43", 44: "Crop 44"
}

# Define the indices for which you need to calculate
indices = [1, 2, 3, 4, 5, 6, 10, 11, 12, 13, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 28, 32, 33, 40, 42, 43, 44]

for i in indices:
    # Create a new column initialized to 0
    df[f'crop_varty_24_{i}'] = 0
    
    # Check if total plots column exists for the crop to avoid KeyError
    if f'tot_plot_24_{i}' in df.columns:
        # Set value to 1 where total plots > 0
        df.loc[df[f'tot_plot_24_{i}'] > 0, f'crop_varty_24_{i}'] = 1

    # Replace missing values with 0 (not strictly necessary unless there are NaNs in crop_varty_24_{i})
    df[f'crop_varty_24_{i}'].fillna(0, inplace=True)
    
    # Calculate percentage
    df[f'crop_varty_per_24_{i}'] = df[f'crop_varty_24_{i}'] * 100


## Rice varieties

###  number of plots for each rice varieties

In [16]:


# Calculate the number of plots for each rice variety
for i in range(1, 4):  # i goes from 1 to 3
    # Create a list of all columns that match the pattern for this rice variety
    columns = [col for col in df.columns if col.startswith(f'rice_var_23_{i}_')]
    
    # Calculate the row total for these columns
    if columns:  # Ensure there are columns to sum
        df[f'tot_rice_plot_{i}'] = df[columns].sum(axis=1)
    else:
        df[f'tot_rice_plot_{i}'] = pd.NA  # Use pd.NA for a more appropriate NA handling in pandas

# Display the new columns or inspect the first few rows to verify
print(df[[f'tot_rice_plot_{i}' for i in range(1, 4)]].head())






   tot_rice_plot_1  tot_rice_plot_2  tot_rice_plot_3
0              0.0              0.0              0.0
1              0.0              0.0              0.0
2              0.0              0.0              0.0
3              0.0              0.0              0.0
4              0.0              0.0              0.0


##  At least one rice variety per household

In [17]:


# Setting flags for households growing at least one rice variety
for i in range(1, 4):  # i goes from 1 to 3, for each of the three rice varieties
    col_name = f'rice_varty_{i}'

    # Initialize the column to 0
    df[col_name] = 0

    # Check if the total plot column exists to avoid KeyError
    if f'tot_rice_plot_{i}' in df.columns:
        # Set to 1 if there are more than 0 plots
        df.loc[df[f'tot_rice_plot_{i}'] > 0, col_name] = 1

# Display the new columns or inspect the first few rows to verify
print(df[[f'rice_varty_{i}' for i in range(1, 4)]].head())


   rice_varty_1  rice_varty_2  rice_varty_3
0             0             0             0
1             0             0             0
2             0             0             0
3             0             0             0
4             0             0             0


## Rice Varaties

In [18]:
# Define the rice varieties and their descriptions
rice_varieties = {
    1: "Short growing variety--90 days",
    2: "Medium growing variety--90-125 days",
    3: "Long growing variety--over 125 days"
}

# Example usage: If you want to display or use these descriptions, you might:
for x in range(1, 4):
    col_name = f'rice_varty_{x}'
    if col_name in df.columns:
        print(f"Description for {col_name}: {rice_varieties[x]}")
        # You might also use these descriptions in plots or reports
        


Description for rice_varty_1: Short growing variety--90 days
Description for rice_varty_2: Medium growing variety--90-125 days
Description for rice_varty_3: Long growing variety--over 125 days


## Input costs and value of crop produced

In [19]:


# Initialize new crop_unit_x columns to default values (None or another placeholder)
for i in range(1, 6):  # Adjust the range if you need more or fewer columns
    df[f'crop_unit_{i}'] = None  # Start with all values as None

    # Ensure column is treated as string
    df[f'crop_unit_name_{i}'] = df[f'crop_unit_name_{i}'].astype(str)

    # Conditions based on crop_unit_name_x columns
    kg_conditions = ['Kilogram', 'kg', 'Kilo', '400', '1', '2']
    quintal_conditions = ['Kuntal', 'Quintal', 'Quntal', 'Kunatal', 'Quintol', 'Kubtal', 'Kwintl', 'Kintel', 'Kentel', 'Kivintal', 'Kintal', 'Kiwintal', 'Kvintal', 'Qunital', 'Kitel', 'Kwintal', '0']
    man_conditions = ['Man', 'man']
    khadi_conditions = ['Khandi', 'Khadi', 'Kh']
    toli_conditions = ['Toli']

    # Applying conditions to assign crop_unit_x values
    df.loc[df[f'crop_unit_name_{i}'].isin(kg_conditions), f'crop_unit_{i}'] = 1
    df.loc[df[f'crop_unit_name_{i}'].str.lower().isin([x.lower() for x in quintal_conditions]), f'crop_unit_{i}'] = 2
    df.loc[df[f'crop_unit_name_{i}'].str.lower().isin([x.lower() for x in man_conditions]), f'crop_unit_{i}'] = 3
    df.loc[df[f'crop_unit_name_{i}'].str.lower().isin([x.lower() for x in khadi_conditions]), f'crop_unit_{i}'] = 4
    df.loc[df[f'crop_unit_name_{i}'].isin(toli_conditions), f'crop_unit_{i}'] = 5

    # Ensure all values are filled, assigning 2 as default if still None
    df[f'crop_unit_{i}'].fillna(2, inplace=True)

# Display the DataFrame to verify changes
print(df[['crop_unit_1', 'crop_unit_2', 'crop_unit_3', 'crop_unit_4', 'crop_unit_5']])


      crop_unit_1  crop_unit_2  crop_unit_3  crop_unit_4  crop_unit_5
0               2            2            3            2            2
1               2            2            3            2            2
2               2            2            3            2            2
3               2            2            3            2            2
4               2            2            3            2            2
...           ...          ...          ...          ...          ...
1041            2            2            2            2            2
1042            2            2            2            2            2
1043            2            2            2            2            2
1044            2            2            2            2            2
1045            2            2            2            2            2

[1046 rows x 5 columns]


In [20]:
# Print unique values in the crop_unit_name columns
for i in range(1, 6):
    print(f"Unique values in crop_unit_name_{i}: {df[f'crop_unit_name_{i}'].unique()}")


Unique values in crop_unit_name_1: ['nan' 'Kg/Kilogram/Kilogram/kilo']
Unique values in crop_unit_name_2: ['nan' 'Quintal/Kuintal/KUINTAL']
Unique values in crop_unit_name_3: ['Man' 'nan']
Unique values in crop_unit_name_4: ['nan']
Unique values in crop_unit_name_5: ['nan' 'Toli']


In [21]:
# Check for a specific condition that might be failing
test_column = 'crop_unit_name_1'
test_values = df[test_column].str.lower().unique()
print("Testing values:", test_values)
print("Quintal conditions check:", any(val in [x.lower() for x in quintal_conditions] for val in test_values))

# This helps to understand whether any of the test_values are actually in your conditions


Testing values: ['nan' 'kg/kilogram/kilogram/kilo']
Quintal conditions check: False


## Households with missing crop unit

In [22]:


# Replace missing crop units for specific households
df.loc[(df['crop_unit_1'].isna()) & (df['ff_code'].isin(["11101_24_M", "12102_18_M", "32203_12_M", "32203_18_M", "32206_4_M"])), 'crop_unit_1'] = 2
df.loc[(df['crop_unit_2'].isna()) & (df['ff_code'] == "32203_18_M"), 'crop_unit_2'] = 2

# Recode 999 to NaN for specific columns
crop_har_amt_cols = ['crop_har_amt_1', 'crop_har_amt_2', 'crop_har_amt_3'] #'crop_har_amt_4', 'crop_har_amt_5', 'crop_har_amt_6']
for col in crop_har_amt_cols:
    df[col] = df[col].replace(999, np.nan)

# Adjust crop harvest amounts for unit 2
for i in range(1, 4):
    df.loc[(df[f'crop_unit_{i}'] == 2) & (df[f'crop_har_amt_{i}'] >= 200) & (~df[f'crop_har_amt_{i}'].isna()), f'crop_har_amt_{i}'] /= 100

# Specific replacements for crop_har_amt_3
df.loc[df['ff_code'] == "12104_23_F", 'crop_har_amt_3'] = 20
df.loc[df['ff_code'] == "12102_26_M", 'crop_har_amt_3'] = 23

# Drop columns related to crop harvest amount in kg
df.drop(columns=[col for col in df.columns if col.startswith('crop_har_amt_kg_')], inplace=True)

# Generate new crop harvest amount in kg based on crop unit
for x in range(1, 4):
    new_col = f'crop_har_amt_kg_{x}'
    df[new_col] = pd.NA  # Initialize new column with NaN

    df.loc[df[f'crop_unit_{x}'] == 1, new_col] = df[f'crop_har_amt_{x}']
    df.loc[df[f'crop_unit_{x}'] == 2, new_col] = df[f'crop_har_amt_{x}'] * 100
    df.loc[df[f'crop_unit_{x}'] == 3, new_col] = df[f'crop_har_amt_{x}'] * 37
    df.loc[df[f'crop_unit_{x}'] == 4, new_col] = df[f'crop_har_amt_{x}'] * 746.48
    df.loc[df[f'crop_unit_{x}'] == 5, new_col] = df[f'crop_har_amt_{x}'] * 0.011




##  HHs with missing yield data because not harvested yet

### Replace outliers

### Crop price

### crop unit price in kg

### generate price for each crop

###

In [23]:


# HHs with missing yield data because not harvested yet
df.loc[(df['crop_har_amt_kg_1'].isna()) & (df['f_crop_var_1'] == 1) & (df['ff_code'] == "31102_6_M"), 'crop_har_amt_kg_1'] = 1800
df.loc[(df['crop_har_amt_kg_1'].isna()) & (df['f_crop_var_1'] == 1) & (df['ff_code'] == "31103_17_F"), 'crop_har_amt_kg_1'] = 842
df.loc[(df['f_crop_var_1'] == 1) & (df['ff_code'].isin(["31110_11_F", "31110_6_M"])), 'crop_har_amt_kg_1'] = 1336
df.loc[(df['f_crop_var_1'] == 23) & (df['ff_code'] == "32206_2_M"), 'crop_har_amt_kg_1'] = 1000
df.loc[(df['f_crop_var_1'] == 1) & (df['ff_code'] == "32210_13_F"), 'crop_har_amt_kg_1'] = 2757

# Recode crop unit prices
price_cols = ['crop_unit_price_', 'crop_unit_exp_price_']
for col in price_cols:
    for i in range(1, 4):
        df[f'{col}{i}'] = df[f'{col}{i}'].replace(999, np.nan)

# Crop unit price in kg
for i in range(1, 4):
    df[f'crop_unit_price_kg_{i}'] = pd.NA
    df[f'crop_unit_exp_price_kg_{i}'] = pd.NA

    df.loc[df[f'crop_unit_{i}'] == 1, f'crop_unit_price_kg_{i}'] = df[f'crop_unit_price_{i}']
    df.loc[df[f'crop_unit_{i}'] == 2, f'crop_unit_price_kg_{i}'] = df[f'crop_unit_price_{i}'] / 100
    df.loc[df[f'crop_unit_{i}'] == 3, f'crop_unit_price_kg_{i}'] = df[f'crop_unit_price_{i}'] / 37
    df.loc[df[f'crop_unit_{i}'] == 4, f'crop_unit_price_kg_{i}'] = df[f'crop_unit_price_{i}'] / 746.48
    df.loc[df[f'crop_unit_{i}'] == 5, f'crop_unit_price_kg_{i}'] = df[f'crop_unit_price_{i}'] / 0.011

    df.loc[df[f'crop_unit_{i}'] == 1, f'crop_unit_exp_price_kg_{i}'] = df[f'crop_unit_exp_price_{i}']
    df.loc[df[f'crop_unit_{i}'] == 2, f'crop_unit_exp_price_kg_{i}'] = df[f'crop_unit_exp_price_{i}'] / 100
    df.loc[df[f'crop_unit_{i}'] == 3, f'crop_unit_exp_price_kg_{i}'] = df[f'crop_unit_exp_price_{i}'] / 37
    df.loc[df[f'crop_unit_{i}'] == 4, f'crop_unit_exp_price_kg_{i}'] = df[f'crop_unit_exp_price_{i}'] / 746.48
    df.loc[df[f'crop_unit_{i}'] == 5, f'crop_unit_exp_price_kg_{i}'] = df[f'crop_unit_exp_price_{i}'] / 0.011

# Generate price for each crop
for i in range(1, 49):
    df[f'crop_price_{i}'] = np.where(df['f_crop_var_1'] == i, df['crop_unit_price_kg_1'], np.nan)
    df[f'crop_price_{i}'] = np.where(df['f_crop_var_2'] == i, df['crop_unit_price_kg_2'], df[f'crop_price_{i}'])
    df[f'crop_price_{i}'] = np.where(df['f_crop_var_3'] == i, df['crop_unit_price_kg_3'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where(df['f_crop_var_4'] == i, df['crop_unit_price_kg_4'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where(df['f_crop_var_5'] == i, df['crop_unit_price_kg_5'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where(df['f_crop_var_6'] == i, df['crop_unit_price_kg_6'], df[f'crop_price_{i}'])

    df[f'crop_price_{i}'] = np.where((df['f_crop_var_1'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_1'], df[f'crop_price_{i}'])
    df[f'crop_price_{i}'] = np.where((df['f_crop_var_2'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_2'], df[f'crop_price_{i}'])
    df[f'crop_price_{i}'] = np.where((df['f_crop_var_3'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_3'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where((df['f_crop_var_4'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_4'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where((df['f_crop_var_5'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_5'], df[f'crop_price_{i}'])
    # df[f'crop_price_{i}'] = np.where((df['f_crop_var_6'] == i) & (df[f'crop_price_{i}'].isna()), df['crop_unit_exp_price_kg_6'], df[f'crop_price_{i}'])

# Display the modified DataFrame
print(df)


              deviceid    device_num  \
0     3ebda66e2e178c42  9.512526e+09   
1     3ebda66e2e178c42  9.512526e+09   
2     26a320911e7e2e98  9.193132e+11   
3     77ba5160422fc308  9.195109e+11   
4     3bc7b409f5c91501           NaN   
...                ...           ...   
1041   869897049607335  9.183030e+11   
1042  3e55166787c11702           NaN   
1043  3e55166787c11702           NaN   
1044  0b33b96b58263a96           NaN   
1045  3e55166787c11702           NaN   

                                             text_audit     state_name  \
0     https://icraf.surveycto.com/view/submission-at...        Gujarat   
1     https://icraf.surveycto.com/view/submission-at...        Gujarat   
2     https://icraf.surveycto.com/view/submission-at...        Gujarat   
3     https://icraf.surveycto.com/view/submission-at...        Gujarat   
4     https://icraf.surveycto.com/view/submission-at...        Gujarat   
...                                                 ...            ...   
1

## Replace Outliers

## crop price

##  crop unit price in kg 

## Generate Price for each crop

##  Replace price with village averge for hhs missing crop price data 

In [24]:


# Define the conditions and replacements for crop_price_11 based on village_name
conditions = {
    31228: 39,
    1115: 49,
    1116: 40,
    1214: 37,
    31250: 30,  # This is to replace a zero value
    31255: 35
}

# Apply the replacements for NaN values and zero values
for village, price in conditions.items():
    df.loc[(df['crop_price_11'].isna() | (df['crop_price_11'] == 0)) & (df['village_name'] == village), 'crop_price_11'] = price

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_11']].head()


,village_name,crop_price_11
0,Bhatavadiya,NaN
1,Bhatavadiya,NaN
2,Bhatavadiya,NaN
3,Bhatavadiya,NaN
4,Bhatavadiya,NaN


## Paddy Rice

In [25]:


# Define village names and conditions for replacements
village_conditions = {
    'Khaidipada': (1318, 15, None),
    'Moskut': (1319, 16, None),
    'Navagam': (31253, 16, 10000),
    'Vadpada': (13117, 15, 100),
    'Zari': (13118, 16, 30),
    'Ambada': (2112, 16, None),
    'Dabhiya': (2114, 19, None),
    'Teriya': (31130, 14, None),
    'Dharmanpur': (3119, 13, None),
    'Kurwari': (31249, 10, None),
    'Sankalpmafi': (31256, 14, None),
    'Bhagwanpur': (31236, 13, None),
    'Mallapur': (31119, 13, None),
    'Teprha': (31258, 13, None),
    'Rampurbrai': (31254, 13, 100),
    'Mahru': (31251, 11, 100),
    'Husenpur Mohammadpur': (31246, 10, None),
    'Bhainsasha': (31237, 13, None),
    'Badgavan': (31234, 11, None),
    'Husainpur Mirdangi': (31245, 10, None),
    'Elashapur agaiyya': (31244, 17, None),
    'Balbhaddarpur': (3122, 16, 50)
}

# Apply the replacements based on the specified conditions
for village_name, details in village_conditions.items():
    village_code, price, threshold = details
    condition = (df['village_name'] == village_code) & (df['crop_price_1'].isna() | ((threshold is not None) and (df['crop_price_1'] > threshold)))
    df.loc[condition, 'crop_price_1'] = price

# Display the DataFrame to verify the changes
print(df[['village_name', 'crop_price_1']].head(1000))


    village_name crop_price_1
0    Bhatavadiya          NaN
1    Bhatavadiya          NaN
2    Bhatavadiya          NaN
3    Bhatavadiya          NaN
4    Bhatavadiya          NaN
..           ...          ...
995          NaN          NaN
996          NaN          NaN
997          NaN          NaN
998          NaN          NaN
999          NaN          NaN

[1000 rows x 2 columns]


## Maize

In [26]:

# Define conditions and replacements for crop_price_4 based on village_name
maize_price_conditions = {
    2114: (13, None),
    31242: (12, None),
    21215: (10, None),
    2219: (15, None),
    31249: (15, None),
    31256: (15, None),
    31119: (16, None),
    31237: (13, None),
    3122: (16, 50),
    3119: (14, 50),
    31254: (14, 50),
    31246: (16, 100)  # Note: Listed twice in your Stata code with different comments
}

# Apply replacements for missing and conditional values
for village_code, (price, threshold) in maize_price_conditions.items():
    # Check if crop_price_4 is NaN or meets the specific conditions
    condition = df['village_name'] == village_code
    specific_condition = df['crop_price_4'].isna() | ((threshold is not None) and (df['crop_price_4'] >= threshold))

    # Apply price based on the general and specific conditions
    df.loc[condition & specific_condition, 'crop_price_4'] = price

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_4']].head(150)


,village_name,crop_price_4
0,Bhatavadiya,NaN
1,Bhatavadiya,NaN
2,Bhatavadiya,NaN
3,Bhatavadiya,NaN
4,Bhatavadiya,NaN
...,...,...
145,Nani dhanej,NaN
146,Nani dhanej,NaN
147,Nani dhanej,NaN
148,Nani dhanej,NaN


## cotton

In [27]:


# Define conditions and replacements for crop_price_2 and crop_price_12 based on village_name
cotton_price_conditions = {
    1318: (60, 100),   # Village Khaidipada, replace if NaN or >= 100
    13117: (53, None), # Village Vadpada, replace if NaN
    31241: (60, 0),    # Village Dhandhalpur, replace if NaN or 0
    1319: (43, 100),   # Village Moskut, replace if NaN or >= 100
    13118: (50, 100)   # Village Zari, replace if NaN or > 100
}

# Apply replacements for missing and conditional values for crop_price_2
for village_code, (price, threshold) in cotton_price_conditions.items():
    condition = (df['village_name'] == village_code) & (df['crop_price_2'].isna() | (df['crop_price_2'] > threshold) if threshold is not None else True)
    df.loc[condition, 'crop_price_2'] = price

# Specific condition for crop_price_12
condition = (df['crop_price_12'] == 600) & (df['village_name'] == 31239)  # Village Bilud
df.loc[condition, 'crop_price_12'] = 60

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_2', 'crop_price_12']].head(150)


,village_name,crop_price_2,crop_price_12
0,Bhatavadiya,NaN,NaN
1,Bhatavadiya,NaN,NaN
2,Bhatavadiya,NaN,NaN
3,Bhatavadiya,NaN,NaN
4,Bhatavadiya,NaN,NaN
...,...,...,...
145,Nani dhanej,NaN,NaN
146,Nani dhanej,NaN,NaN
147,Nani dhanej,NaN,NaN
148,Nani dhanej,NaN,NaN


## Redgram

In [28]:


# Define conditions and replacements for crop_price_5 and others based on village_name
redgram_price_conditions = {
    31253: (61, None),  # Navagam
    13117: (36, None),  # Vadpada
    31241: (32, None),  # Dhandhalpur
    2219: (35, None),   # Paldha
    31249: (35, None),  # Kurwari
    1318: (60, 100, 20)  # Khaidipada, check if >= 100 or <= 20
}

# Apply replacements for crop_price_5
for village_code, details in redgram_price_conditions.items():
    if len(details) == 2:
        price, threshold = details
        condition = (df['village_name'] == village_code) & (df['crop_price_5'].isna() | ((threshold is not None) and (df['crop_price_5'] >= threshold)))
    else:
        price, upper_threshold, lower_threshold = details
        condition = (df['village_name'] == village_code) & (df['crop_price_5'].isna() | (df['crop_price_5'] >= upper_threshold) | (df['crop_price_5'] <= lower_threshold))
    df.loc[condition, 'crop_price_5'] = price

# Specific conditions for crop_price_6 and crop_price_11
df.loc[(df['crop_price_6'] == 800) & (df['village_name'] == 1319), 'crop_price_6'] = 40  # Moskut
df.loc[(df['crop_price_11'] == 120) & (df['village_name'] == 31253), 'crop_price_11'] = 25  # Navagam

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_5', 'crop_price_6', 'crop_price_11']].head(150)


,village_name,crop_price_5,crop_price_6,crop_price_11
0,Bhatavadiya,NaN,NaN,NaN
1,Bhatavadiya,NaN,NaN,NaN
2,Bhatavadiya,NaN,NaN,NaN
3,Bhatavadiya,NaN,NaN,NaN
4,Bhatavadiya,NaN,NaN,NaN
...,...,...,...,...
145,Nani dhanej,NaN,NaN,NaN
146,Nani dhanej,NaN,NaN,NaN
147,Nani dhanej,NaN,NaN,NaN
148,Nani dhanej,NaN,NaN,NaN


## chillies

In [29]:


# Chillies price conditions based on specific village codes
chillies_price_conditions = {
    14115: 15,  # Gadh
    31235: 15   # Bedaniya
}

# Apply replacements for crop_price_3 if values are missing
for village_code, price in chillies_price_conditions.items():
    condition = (df['village_name'] == village_code) & df['crop_price_3'].isna()
    df.loc[condition, 'crop_price_3'] = price

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_3']].head(150)


,village_name,crop_price_3
0,Bhatavadiya,NaN
1,Bhatavadiya,NaN
2,Bhatavadiya,NaN
3,Bhatavadiya,NaN
4,Bhatavadiya,NaN
...,...,...
145,Nani dhanej,NaN
146,Nani dhanej,NaN
147,Nani dhanej,NaN
148,Nani dhanej,NaN


##  Jowar 

In [30]:


# Update crop_price_13 based on village codes and conditions
village_conditions_13 = {
    14115: 13,  # Gadh
    31238: 13   # Gadh
}

# Apply replacements for crop_price_13 if values are missing or meet specific conditions
for village_code, price in village_conditions_13.items():
    condition = (df['village_name'] == village_code) & df['crop_price_13'].isna()
    df.loc[condition, 'crop_price_13'] = price

# Conditions where price needs to be set based on other criteria
df.loc[(df['crop_price_13'] >= 100) & (df['village_name'].isin([31241, 31243])), 'crop_price_13'] = 35  # Dhandhalpur & Dhinkwali

# Specific replacements for crop_price_24 and crop_price_22
df.loc[(df['crop_price_24'] == 500) & (df['village_name'] == 31253), 'crop_price_24'] = 3.5  # Navagam
df.loc[(df['crop_price_22'] == 15000) & (df['village_name'] == 31251), 'crop_price_22'] = 90  # Mahru

# Display the DataFrame to verify the changes
df[['village_name', 'crop_price_13', 'crop_price_24', 'crop_price_22']].head(150)


,village_name,crop_price_13,crop_price_24,crop_price_22
0,Bhatavadiya,NaN,NaN,NaN
1,Bhatavadiya,NaN,NaN,NaN
2,Bhatavadiya,NaN,NaN,NaN
3,Bhatavadiya,NaN,NaN,NaN
4,Bhatavadiya,NaN,NaN,NaN
...,...,...,...,...
145,Nani dhanej,NaN,NaN,NaN
146,Nani dhanej,NaN,NaN,NaN
147,Nani dhanej,NaN,NaN,NaN
148,Nani dhanej,NaN,NaN,NaN


## soyabean

In [31]:


# Update crop_price_12 based on village codes for soybean
soybean_price_conditions = {
    2128: 51,   # Goradiya
    21215: 56  # Rajola
}

# Apply replacements for crop_price_12 if values are missing
for village_code, price in soybean_price_conditions.items():
    condition = (df['village_name'] == village_code) & df['crop_price_12'].isna()
    df.loc[condition, 'crop_price_12'] = price

# Specific conditions for crop_price_5 and crop_price_11 based on state and exact current values
df.loc[(df['crop_price_5'] == 500) & (df['state_name'] == 1), 'crop_price_5'] = 50  # Condition for state 1
df.loc[(df['crop_price_11'] == 250) & (df['state_name'] == 1), 'crop_price_11'] = 30  # Condition for state 1

# Display the DataFrame to verify the changes
df[['village_name', 'state_name', 'crop_price_12', 'crop_price_5', 'crop_price_11']].head()


,village_name,state_name,crop_price_12,crop_price_5,crop_price_11
0,Bhatavadiya,Gujarat,NaN,NaN,NaN
1,Bhatavadiya,Gujarat,NaN,NaN,NaN
2,Bhatavadiya,Gujarat,NaN,NaN,NaN
3,Bhatavadiya,Gujarat,NaN,NaN,NaN
4,Bhatavadiya,Gujarat,NaN,NaN,NaN


## Replace Outliers

In [32]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Winsorizing crop_price_1 for state_name == 3
df.loc[df['state_name'] == 3, 'crop_price_1'] = winsorize_series(df.loc[df['state_name'] == 3, 'crop_price_1'], 0.01, 0.99)

# Winsorizing crop_price_2 for state_name == 1
df.loc[df['state_name'] == 1, 'crop_price_2'] = winsorize_series(df.loc[df['state_name'] == 1, 'crop_price_2'], 0.01, 0.99)

# Winsorizing crop_price_4 for state_name == 3
df.loc[df['state_name'] == 3, 'crop_price_4'] = winsorize_series(df.loc[df['state_name'] == 3, 'crop_price_4'], 0.01, 0.99)

# Display the DataFrame to verify the changes
df[['state_name', 'crop_price_1', 'crop_price_2', 'crop_price_4']].head()



,state_name,crop_price_1,crop_price_2,crop_price_4
0,Gujarat,NaN,NaN,NaN
1,Gujarat,NaN,NaN,NaN
2,Gujarat,NaN,NaN,NaN
3,Gujarat,NaN,NaN,NaN
4,Gujarat,NaN,NaN,NaN


##  value of crop poduced  - kharif 2023

In [33]:


# Initialize columns for the value of crop produced
for i in range(1, 7):
    df[f'crop_har_val_{i}'] = 0

# Calculate the value of harvested crops based on crop prices and amounts in kg
for i in range(1, 49):  # Adjust the range if the actual number of crops is different
    for j in range(1, 4):
        price_column = f'crop_price_{i}'
        if price_column in df.columns:
            condition = (df[f'f_crop_var_{j}'] == i)
            df.loc[condition, f'crop_har_val_{j}'] = df[f'crop_har_amt_kg_{j}'] * df[price_column]

# Calculate the total value of crops harvested
df['crop_har_val_tot'] = df[[f'crop_har_val_{i}' for i in range(1, 4)]].sum(axis=1)

# Assign a label to the 'crop_har_val_tot' column (optional in pandas, typically for presentation)
df['crop_har_val_tot'].rename("Total value of crops harvested (Kharif 2021)")

# Display the DataFrame to verify the changes
df[['crop_har_val_tot'] + [f'crop_har_val_{i}' for i in range(1, 4)]].head()


,crop_har_val_tot,crop_har_val_1,crop_har_val_2,crop_har_val_3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


##  replace outliers 

In [34]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Winsorizing 'crop_har_val_tot' based on state conditions
state_conditions = [1, 2, 3]  # List states that need winsorization
for state_name in state_conditions:
    # Selecting the part of the DataFrame that matches the state condition
    mask = df['state_name'] == state_name
    df.loc[mask, 'crop_har_val_tot'] = winsorize_series(df.loc[mask, 'crop_har_val_tot'], 0.01, 0.99)

# Display the DataFrame to verify the changes
df[['state_name', 'crop_har_val_tot']].head(150)


,state_name,crop_har_val_tot
0,Gujarat,0
1,Gujarat,0
2,Gujarat,0
3,Gujarat,0
4,Gujarat,0
...,...,...
145,Gujarat,0
146,Gujarat,0
147,Gujarat,0
148,Gujarat,0


### Income from crop production
### Variable costs  


In [35]:


# List of variable names involved in the operation
variable_costs = [
    'f_ld_prep_labour_', 'f_prep_eq_hire_cst_', 'f_pre_treat_labour_',
    'f_pre_treat_equip_amt_', 'f_pre_treat_exp_amt', 'f_irr_pay_rps',
    'f_weed_exp_amt', 'f_weed_con_labour', 'f_weed_con_equip_amt',
    'f_pest_con_exp_amt', 'f_pest_con_labour', 'f_pest_con_equip_amt',
    'f_treat_exp_amt', 'f_treat_labour', 'f_treat_equip_amt',
    'f_seed_exp_amt', 'sd_trt_exp_amt', 'sd_trt_labour', 'f_sow_eq_hire_cst',
    'f_sow_labour', 'f_har_labour', 'f_har_eq_hire_cst', 'f_har_tran_lab_exp',
    'f_har_tran_p_exp'
]

# Apply the recoding for each variable in the list if they exist in the DataFrame
for base_name in variable_costs:
    # Depending on your data, these variables might have indices or similar suffixes
    # Ensure to adjust the loop to handle such cases if they exist
    for i in range(1, 7):  # Assuming there are up to 6 variants of each variable
        var_name = f"{base_name}{i}"
        if var_name in df.columns:
            df[var_name] = df[var_name].replace(999, np.nan)

# Display some of the updated data to verify changes
df[[var for var in df.columns if var.startswith('f_ld_prep_labour_') or var.startswith('f_prep_eq_hire_cst_')]].head()


,f_ld_prep_labour_1,f_prep_eq_hire_cst_1,f_ld_prep_labour_2,f_prep_eq_hire_cst_2,f_ld_prep_labour_3,f_prep_eq_hire_cst_3,f_ld_prep_labour_4,f_prep_eq_hire_cst_4,f_ld_prep_labour_5,f_prep_eq_hire_cst_5,...,f_ld_prep_labour_24_1,f_prep_eq_hire_cst_24_1,f_ld_prep_labour_24_2,f_prep_eq_hire_cst_24_2,f_ld_prep_labour_24_3,f_prep_eq_hire_cst_24_3,f_ld_prep_labour_24_4,f_prep_eq_hire_cst_24_4,f_ld_prep_labour_24_5,f_prep_eq_hire_cst_24_5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3000.0,NaN,3000.0,NaN,2000.0,NaN,NaN,NaN,NaN,...,NaN,3500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,5000.0,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2400.0,NaN,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Assume labour as overhead cost proxied by the maximum of labour expense for all activities 

## replcae outliers 


In [36]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# List of labor-related cost variables
labor_vars = [
    'f_ld_prep_labour_', 'f_pre_treat_labour_', 'f_weed_con_labour',
    'f_pest_con_labour', 'f_treat_labour', 'sd_trt_labour', 'f_sow_labour',
    'f_har_labour', 'f_har_tran_lab_exp'
]

# Winsorizing labor cost variables for each state condition
states = [1, 2, 3]
for state_name in states:
    for base_name in labor_vars:
        for i in range(1, 7):  # Assuming there are up to 6 variants of each variable
            var_name = f"{base_name}{i}"
            if var_name in df.columns:
                mask = (df['state_name'] == state_name)
                df.loc[mask, var_name] = winsorize_series(df.loc[mask, var_name], 0.01, 0.99)

# Calculate the maximum labor expense across all specified activities
max_columns = [f"{var}{i}" for var in labor_vars for i in range(1, 7) if f"{var}{i}" in df.columns]
df['labor_total'] = df[max_columns].max(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'labor_total'] + max_columns].head()


,state_name,labor_total,f_ld_prep_labour_1,f_ld_prep_labour_2,f_ld_prep_labour_3,f_ld_prep_labour_4,f_ld_prep_labour_5,f_ld_prep_labour_6,f_pre_treat_labour_1,f_pre_treat_labour_2,f_pre_treat_labour_3,f_pre_treat_labour_4,f_pre_treat_labour_5,f_pre_treat_labour_6
0,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  Land preparation  

In [37]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame

def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# List of equipment hire cost variables for land preparation
equipment_hire_vars = [col for col in df.columns if 'f_prep_eq_hire_cst_' in col]

# Winsorizing equipment hire cost variables for each state condition
states = [1, 2, 3]  # Adjust these as per your state_name values, e.g., 'State 1', 'State 2', 'State 3'
for state in states:
    for var in equipment_hire_vars:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for land preparation
df['landprep_exp'] = df[equipment_hire_vars].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'landprep_exp'] + equipment_hire_vars].head(150)


,state_name,landprep_exp,f_prep_eq_hire_cst_1,f_prep_eq_hire_cst_2,f_prep_eq_hire_cst_3,f_prep_eq_hire_cst_4,f_prep_eq_hire_cst_5,f_prep_eq_hire_cst_6,f_prep_eq_hire_cst_24_1,f_prep_eq_hire_cst_24_2,f_prep_eq_hire_cst_24_3,f_prep_eq_hire_cst_24_4,f_prep_eq_hire_cst_24_5
0,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,11500.0,3000.0,3000.0,2000.0,NaN,NaN,NaN,3500.0,NaN,NaN,NaN,NaN
3,Gujarat,7000.0,5000.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,6100.0,2400.0,1200.0,NaN,NaN,NaN,NaN,2500.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,Gujarat,600.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,Gujarat,5400.0,3000.0,NaN,NaN,NaN,NaN,NaN,2400.0,NaN,NaN,NaN,NaN
148,Gujarat,5000.0,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Land preparation

In [38]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# List of equipment hire cost variables for land preparation
equipment_hire_vars = [col for col in df.columns if 'f_prep_eq_hire_cst_' in col]

# Winsorizing equipment hire cost variables for each state condition
states = [1, 2, 3]  # Adjust these as per your state_name values, e.g., 'State 1', 'State 2', 'State 3'
for state in states:
    for var in equipment_hire_vars:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for land preparation
df['landprep_exp'] = df[equipment_hire_vars].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'landprep_exp'] + equipment_hire_vars].head()


,state_name,landprep_exp,f_prep_eq_hire_cst_1,f_prep_eq_hire_cst_2,f_prep_eq_hire_cst_3,f_prep_eq_hire_cst_4,f_prep_eq_hire_cst_5,f_prep_eq_hire_cst_6,f_prep_eq_hire_cst_24_1,f_prep_eq_hire_cst_24_2,f_prep_eq_hire_cst_24_3,f_prep_eq_hire_cst_24_4,f_prep_eq_hire_cst_24_5
0,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,11500.0,3000.0,3000.0,2000.0,NaN,NaN,NaN,3500.0,NaN,NaN,NaN,NaN
3,Gujarat,7000.0,5000.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,6100.0,2400.0,1200.0,NaN,NaN,NaN,NaN,2500.0,NaN,NaN,NaN,NaN


##  pre-sowing soil treatment - labour equipment and substances (e.g. fertilizer)

In [39]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns for labor, equipment, and expenses
labour_columns = [col for col in df.columns if 'f_pre_treat_labour_' in col]
equip_amount_columns = [col for col in df.columns if 'f_pre_treat_equip_amt_' in col]
expense_amount_columns = [col for col in df.columns if 'f_pre_treat_exp_amt' in col]

# Calculate the maximum labor cost for pre-sowing soil treatment
df['pre_treat_lab'] = df[labour_columns].max(axis=1)

# Winsorizing equipment and expense amounts for each state condition
states = [1, 2, 3]
for state in states:
    for column in equip_amount_columns + expense_amount_columns:
        mask = df['state_name'] == state
        df.loc[mask, column] = winsorize_series(df.loc[mask, column], 0.01, 0.99)

# Sum equipment and expense amounts to calculate total pre-treatment expenses
df['pre_treatment_exp'] = df[equip_amount_columns + expense_amount_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'pre_treat_lab', 'pre_treatment_exp'] + labour_columns + equip_amount_columns + expense_amount_columns].head()


,state_name,pre_treat_lab,pre_treatment_exp,f_pre_treat_labour_1,f_pre_treat_labour_2,f_pre_treat_labour_3,f_pre_treat_labour_4,f_pre_treat_labour_5,f_pre_treat_labour_6,f_pre_treat_labour_24_1,...,f_pre_treat_exp_amt_2,f_pre_treat_exp_amt_3,f_pre_treat_exp_amt_4,f_pre_treat_exp_amt_5,f_pre_treat_exp_amt_6,f_pre_treat_exp_amt_24_1,f_pre_treat_exp_amt_24_2,f_pre_treat_exp_amt_24_3,f_pre_treat_exp_amt_24_4,f_pre_treat_exp_amt_24_5
0,Gujarat,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,NaN,8500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2000.0,2500.0,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,NaN
3,Gujarat,NaN,13000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,NaN,480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,480.0,NaN,NaN,NaN,NaN


## Irrigation costs

In [40]:

def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to irrigation payments
irrigation_payment_vars = [col for col in df.columns if 'f_irr_pay_rps' in col]

# Winsorizing irrigation payment variables for each state condition
states = [1, 2, 3]  # Adjust these as per your state_name values
for state in states:
    for var in irrigation_payment_vars:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for irrigation
df['irrg_exp'] = df[irrigation_payment_vars].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'irrg_exp'] + irrigation_payment_vars].head()


,state_name,irrg_exp,f_irr_pay_rps_1,f_irr_pay_rps_2,f_irr_pay_rps_3,f_irr_pay_rps_4,f_irr_pay_rps_5,f_irr_pay_rps_6,f_irr_pay_rps_24_1,f_irr_pay_rps_24_2,f_irr_pay_rps_24_3,f_irr_pay_rps_24_4,f_irr_pay_rps_24_5
0,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,7300.0,1500.0,NaN,2800.0,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,NaN
3,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,4900.0,1000.0,1500.0,NaN,NaN,NaN,NaN,2400.0,NaN,NaN,NaN,NaN


##  Weed control - herbicides, labour and equipment hire 

In [41]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to weed control labor and expenses
labor_columns = [col for col in df.columns if 'f_weed_con_labour' in col]
expense_amount_columns = [col for col in df.columns if 'f_weed_exp_amt' in col]
equip_amount_columns = [col for col in df.columns if 'f_weed_con_equip_amt' in col]

# Calculate the maximum labor cost for weed control
df['weed_lab'] = df[labor_columns].max(axis=1)

# Winsorizing expense and equipment amount variables for each state condition
states = [1, 2, 3]  # Ensure these match your actual state_name values
for state in states:
    for var in expense_amount_columns + equip_amount_columns:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for weed control
df['weed_exp'] = df[expense_amount_columns + equip_amount_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'weed_lab', 'weed_exp'] + labor_columns + expense_amount_columns + equip_amount_columns].head()


,state_name,weed_lab,weed_exp,f_weed_con_labour_1,f_weed_con_labour_2,f_weed_con_labour_3,f_weed_con_labour_4,f_weed_con_labour_5,f_weed_con_labour_6,f_weed_con_labour_24_1,...,f_weed_con_equip_amt_2,f_weed_con_equip_amt_3,f_weed_con_equip_amt_4,f_weed_con_equip_amt_5,f_weed_con_equip_amt_6,f_weed_con_equip_amt_24_1,f_weed_con_equip_amt_24_2,f_weed_con_equip_amt_24_3,f_weed_con_equip_amt_24_4,f_weed_con_equip_amt_24_5
0,Gujarat,200000.0,0.0,200000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gujarat,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## pest control - pesticides, labour, equipment 

In [42]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to pest control expenses and equipment
pesticide_expense_columns = [col for col in df.columns if 'f_pest_con_exp_amt' in col]
pesticide_equipment_columns = [col for col in df.columns if 'f_pest_con_equip_amt' in col]

# Winsorizing pest control expense and equipment amount variables for each state condition
states = [1, 2, 3]  # Adjust these as per your actual state_name values
for state in states:
    for var in pesticide_expense_columns + pesticide_equipment_columns:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# You might want to calculate the total expenses for pest control (sum of expenses and equipment)
df['pest_control_total'] = df[pesticide_expense_columns + pesticide_equipment_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'pest_control_total'] + pesticide_expense_columns + pesticide_equipment_columns].tail()


,state_name,pest_control_total,f_pest_con_exp_amt_1,f_pest_con_exp_amt_2,f_pest_con_exp_amt_3,f_pest_con_exp_amt_4,f_pest_con_exp_amt_5,f_pest_con_exp_amt_6,f_pest_con_exp_amt_24_1,f_pest_con_exp_amt_24_2,...,f_pest_con_equip_amt_2,f_pest_con_equip_amt_3,f_pest_con_equip_amt_4,f_pest_con_equip_amt_5,f_pest_con_equip_amt_6,f_pest_con_equip_amt_24_1,f_pest_con_equip_amt_24_2,f_pest_con_equip_amt_24_3,f_pest_con_equip_amt_24_4,f_pest_con_equip_amt_24_5
1041,Uttar Pradesh,500.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1042,Uttar Pradesh,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1043,Uttar Pradesh,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1044,Uttar Pradesh,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1045,Uttar Pradesh,2000.0,2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pest Control Costs Calculation

In [43]:

# Identify the maximum labor costs for pest control across potentially multiple columns
labor_columns = [col for col in df.columns if 'f_pest_con_labour' in col]
df['pest_con_labour'] = df[labor_columns].max(axis=1)

# Identify columns related to pest control expenses and equipment
expense_columns = [col for col in df.columns if 'f_pest_con_exp_amt' in col]
equipment_columns = [col for col in df.columns if 'f_pest_con_equip_amt' in col]

# Calculate the total expenses for pest control by summing the expenses and equipment costs
df['pest_exp'] = df[expense_columns + equipment_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['pest_con_labour', 'pest_exp'] + labor_columns + expense_columns + equipment_columns].head()


,pest_con_labour,pest_exp,f_pest_con_labour_1,f_pest_con_labour_2,f_pest_con_labour_3,f_pest_con_labour_4,f_pest_con_labour_5,f_pest_con_labour_6,f_pest_con_labour_24_1,f_pest_con_labour_24_2,...,f_pest_con_equip_amt_2,f_pest_con_equip_amt_3,f_pest_con_equip_amt_4,f_pest_con_equip_amt_5,f_pest_con_equip_amt_6,f_pest_con_equip_amt_24_1,f_pest_con_equip_amt_24_2,f_pest_con_equip_amt_24_3,f_pest_con_equip_amt_24_4,f_pest_con_equip_amt_24_5
0,25000.0,160000.0,25000.0,25000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,60000.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,9000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Growth stimulants

In [44]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to growth stimulant expenses and equipment
stimulant_expense_columns = [col for col in df.columns if 'f_treat_exp_amt' in col]
stimulant_equipment_columns = [col for col in df.columns if 'f_treat_equip_amt' in col]

# Winsorizing stimulant expense and equipment amount variables for each state condition
states = [1, 2, 3]  # Adjust these as per your actual state_name values
for state in states:
    for var in stimulant_expense_columns + stimulant_equipment_columns:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for growth stimulants
df['stimulant_exp'] = df[stimulant_expense_columns + stimulant_equipment_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'stimulant_exp'] + stimulant_expense_columns + stimulant_equipment_columns].head()


,state_name,stimulant_exp,f_treat_exp_amt_1,f_treat_exp_amt_2,f_treat_exp_amt_3,f_treat_exp_amt_4,f_treat_exp_amt_5,f_treat_exp_amt_6,f_treat_exp_amt_24_1,f_treat_exp_amt_24_2,...,f_treat_equip_amt_2,f_treat_equip_amt_3,f_treat_equip_amt_4,f_treat_equip_amt_5,f_treat_equip_amt_6,f_treat_equip_amt_24_1,f_treat_equip_amt_24_2,f_treat_equip_amt_24_3,f_treat_equip_amt_24_4,f_treat_equip_amt_24_5
0,Gujarat,450000.0,250000.0,200000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## seed expenses

In [45]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to seed expenses, seed treatment expenses, and equipment hire costs
seed_expense_columns = [col for col in df.columns if 'f_seed_exp_amt' in col]
seed_treatment_expense_columns = [col for col in df.columns if 'sd_trt_exp_amt' in col]
equipment_hire_columns = [col for col in df.columns if 'f_sow_eq_hire_cst' in col]

# Winsorizing variables for each state condition
states = [1, 2, 3]  # Adjust these as per your actual state_name values
for state in states:
    for var in seed_expense_columns + seed_treatment_expense_columns + equipment_hire_columns:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for seed related costs
df['seed_exp'] = df[seed_expense_columns + seed_treatment_expense_columns + equipment_hire_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'seed_exp'] + seed_expense_columns + seed_treatment_expense_columns + equipment_hire_columns].head()


,state_name,seed_exp,f_seed_exp_amt_1,f_seed_exp_amt_2,f_seed_exp_amt_3,f_seed_exp_amt_24_1,f_seed_exp_amt_24_2,f_seed_exp_amt_24_3,f_seed_exp_amt_24_4,sd_trt_exp_amt_1,...,sd_trt_exp_amt_24_2,sd_trt_exp_amt_24_3,sd_trt_exp_amt_24_4,f_sow_eq_hire_cst_1,f_sow_eq_hire_cst_2,f_sow_eq_hire_cst_3,f_sow_eq_hire_cst_24_1,f_sow_eq_hire_cst_24_2,f_sow_eq_hire_cst_24_3,f_sow_eq_hire_cst_24_4
0,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,80000.0,80000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,6000.0,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gujarat,50000.0,50000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5000.0,NaN,NaN,NaN


##  Harvest and transport costs

In [46]:


def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Identify columns related to harvest equipment hire costs and transport expenses
harvest_equipment_hire_columns = [col for col in df.columns if 'f_har_eq_hire_cst' in col]
transport_expense_columns = [col for col in df.columns if 'f_har_tran_p_exp' in col]

# Winsorizing variables for each state condition
states = [1, 2, 3]  # Adjust these as per your actual state_name values
for state in states:
    for var in harvest_equipment_hire_columns + transport_expense_columns:
        mask = (df['state_name'] == state)
        df.loc[mask, var] = winsorize_series(df.loc[mask, var], 0.01, 0.99)

# Calculate the total expenses for harvest and transportation
df['har_trans_exp'] = df[harvest_equipment_hire_columns + transport_expense_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['state_name', 'har_trans_exp'] + harvest_equipment_hire_columns + transport_expense_columns].head()


,state_name,har_trans_exp,f_har_eq_hire_cst_1,f_har_eq_hire_cst_2,f_har_eq_hire_cst_3,f_har_eq_hire_cst_24_1,f_har_eq_hire_cst_24_2,f_har_eq_hire_cst_24_3,f_har_eq_hire_cst_24_4,f_har_tran_p_exp_1,f_har_tran_p_exp_2,f_har_tran_p_exp_3,f_har_tran_p_exp_24_1,f_har_tran_p_exp_24_2,f_har_tran_p_exp_24_3,f_har_tran_p_exp_24_4
0,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gujarat,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##  Variable costs minus labor cost 

In [47]:


# Calculate the total of various variable costs excluding labor
cost_columns = [
    'landprep_exp', 'pre_treatment_exp', 'irrg_exp', 'weed_exp',
    'pest_exp', 'stimulant_exp', 'seed_exp', 'har_trans_exp'
]

# Ensure all columns exist in DataFrame and are filled with zeros if not present
for column in cost_columns:
    if column not in df.columns:
        df[column] = 0

# Summing up the variable costs to a new column
df['varcost_othrs'] = df[cost_columns].sum(axis=1)

# Display the DataFrame to verify the changes
df[['varcost_othrs'] + cost_columns].head()


,varcost_othrs,landprep_exp,pre_treatment_exp,irrg_exp,weed_exp,pest_exp,stimulant_exp,seed_exp,har_trans_exp
0,610000.0,0.0,0.0,0.0,0.0,160000.0,450000.0,0.0,0.0
1,140000.0,0.0,0.0,0.0,0.0,60000.0,0.0,80000.0,0.0
2,33300.0,11500.0,8500.0,7300.0,0.0,0.0,0.0,6000.0,0.0
3,81000.0,7000.0,13000.0,0.0,2000.0,9000.0,0.0,50000.0,0.0
4,16480.0,6100.0,480.0,4900.0,0.0,0.0,0.0,5000.0,0.0


##  variables costs including labor

## Net Return

In [48]:

def winsorize_series(series, lower_percentile, upper_percentile):
    """
    Apply winsorization to a pandas Series by clipping values to specified percentiles.
    """
    quantiles = series.quantile([lower_percentile, upper_percentile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# Calculate the total variable costs including labor
df['varcost_total'] = df['labor_total'] + df['varcost_othrs']

# Calculate the net farm return
df['farm_return'] = df['crop_har_val_tot'] - df['varcost_total']

# Winsorizing the farm return variable for each state condition
states = [1, 2, 3]  # Adjust these as per your actual state_name values
for state in states:
    mask = (df['state_name'] == state)
    df.loc[mask, 'farm_return'] = winsorize_series(df.loc[mask, 'farm_return'], 0.01, 0.99)

# Adding labels to variables (this step is conceptual as pandas does not use variable labels like Stata)
df['farm_return'].attrs['description'] = "Net farm return (‚Çπ) Kharif 2021"
df['crop_har_val_tot'].attrs['description'] = "Value of crop produced (‚Çπ) Kharif 2021"

# Display the DataFrame to verify the changes
df[['state_name', 'farm_return', 'crop_har_val_tot', 'varcost_total']].head()


,state_name,farm_return,crop_har_val_tot,varcost_total
0,Gujarat,NaN,0,NaN
1,Gujarat,NaN,0,NaN
2,Gujarat,NaN,0,NaN
3,Gujarat,NaN,0,NaN
4,Gujarat,NaN,0,NaN


##  Adoption of LCA practices 
## Land preparation method used 


In [49]:

# Iterate over each of the six fields/crops
for i in range(1, 7):
    prep_method_this_year = f'f_ld_prep_{i}'
    prep_method_last_year = f'f_prep_24_how_{i}'
    same_prep_flag = f'f_prep_24_same_{i}'
    
    # Check if columns exist to avoid KeyError
    if prep_method_this_year in df.columns and prep_method_last_year in df.columns:
        # Update the flag based on whether the preparation method stayed the same
        df[same_prep_flag] = (df[prep_method_this_year] == df[prep_method_last_year]).astype(int)
    else:
        # Optionally handle cases where columns might not exist
        df[same_prep_flag] = 0  # Default to 0 or another placeholder if necessary

# Display the DataFrame to verify the changes
df[[f'f_prep_24_same_{i}' for i in range(1, 7)]].head()


,f_prep_24_same_1,f_prep_24_same_2,f_prep_24_same_3,f_prep_24_same_4,f_prep_24_same_5,f_prep_24_same_6
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


##  The same land preparation method used in kharif 2024

In [50]:


# Update land preparation methods if they remained the same
for i in range(1, 6):  # for each type of preparation
    for j in range(1, 7):  # for each field or crop
        current_prep_col = f'f_ld_prep_{i}_{j}'
        last_year_prep_col = f'f_prep_24_how_{i}_{j}'
        same_prep_col = f'f_prep_24_same_{j}'

        # Update previous year's preparation method if it was the same
        df.loc[df[same_prep_col] == 1, last_year_prep_col] = df[current_prep_col]

# Calculate total land preparation for current and previous years for each type
for i in range(1, 6):
    current_prep_cols = [f'f_ld_prep_{i}_{j}' for j in range(1, 7) if f'f_ld_prep_{i}_{j}' in df.columns]
    last_year_prep_cols = [f'f_prep_24_how_{i}_{j}' for j in range(1, 7) if f'f_prep_24_how_{i}_{j}' in df.columns]

    # Using sum across the rows for each set of columns
    df[f'tot_landprep_{i}'] = df[current_prep_cols].sum(axis=1)
    df[f'tot_landprep_24_{i}'] = df[last_year_prep_cols].sum(axis=1)

# Output the relevant columns to verify
df[[f'tot_landprep_{i}' for i in range(1, 6)] + [f'tot_landprep_24_{i}' for i in range(1, 6)]].head()


,tot_landprep_1,tot_landprep_2,tot_landprep_3,tot_landprep_4,tot_landprep_5,tot_landprep_24_1,tot_landprep_24_2,tot_landprep_24_3,tot_landprep_24_4,tot_landprep_24_5
0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


##  Land preparation method used on at least 1 plot 

In [51]:
import pandas as pd

# DataFrame `df` should already contain columns like `tot_landprep_1` to `tot_landprep_5` for current year and `tot_landprep_24_1` to `tot_landprep_24_5` for previous year.

# Definitions (Dictionary can be used for easier maintenance)
land_prep_methods = {
    1: "Animal drawn plough",
    2: "Tractor or other machine like disc plough",
    3: "Turning of soil with handheld hoe",
    4: "Minimum or reduced tillage",
    5: "Zero tillage"
}

# Initialize columns for land preparation presence
for i in range(1, 6):
    df[f'land_prep_{i}'] = 0
    df[f'land_prep_24_{i}'] = 0

    # Update flags based on totals
    df.loc[df[f'tot_landprep_{i}'] > 0, f'land_prep_{i}'] = 1
    df.loc[df[f'tot_landprep_24_{i}'] > 0, f'land_prep_24_{i}'] = 1

    # Assigning labels
    df[f'land_prep_{i}'].rename(land_prep_methods[i])
    df[f'land_prep_24_{i}'].rename(land_prep_methods[i])

# Create specific variables for each method
df['min_til_23'] = df['land_prep_4']
df['min_til_24'] = df['land_prep_24_4']
df['no_til_23'] = df['land_prep_5']
df['no_til_24'] = df['land_prep_24_5']
df['animal_plough_23'] = df['land_prep_1']
df['animal_plough_24'] = df['land_prep_24_1']
df['tractor_plough_23'] = df['land_prep_2']
df['tractor_plough_24'] = df['land_prep_24_2']
df['handheld_hoe_23'] = df['land_prep_3']
df['handheld_hoe_24'] = df['land_prep_24_3']

# Optionally, print to verify
print(df[['min_til_23', 'no_til_23', 'animal_plough_23', 'tractor_plough_23', 'handheld_hoe_23']].head())


   min_til_23  no_til_23  animal_plough_23  tractor_plough_23  handheld_hoe_23
0           0          0                 0                  1                0
1           0          0                 0                  1                0
2           0          0                 0                  1                0
3           0          0                 0                  1                0
4           1          0                 0                  1                0


##  land preparation more than once using tractor or any other machine

In [52]:

# Column names that might exist for tractor preparation across multiple instances
# This list should be adjusted to reflect actual column names in your DataFrame
tractor_columns = [col for col in df.columns if 'trac_prep_1p_n_' in col]

# Using the `max` function across the specified row axis to determine the maximum value of tractor preparation indicators
df['trac_prep_1p'] = df[tractor_columns].max(axis=1)

# Optionally add a label to the new column if required (This is more of a documentation in pandas, not a label as in Stata)
df['trac_prep_1p'].rename("Land preparation more than once using tractor or machine", inplace=True)

# Print to verify the output
print(df[['trac_prep_1p']].head())


   trac_prep_1p
0           5.0
1           3.0
2           1.0
3           3.0
4           2.0


##  Soil  treatment 
## pre-sowing treatment 
##  Total amount of fertilizer applied 


In [53]:

# Ensure all relevant columns are numeric
for col in df.columns:
    if col.startswith('f_pre_treat_typ_') or col.startswith('f_pre_treat_typ_24_') or col.startswith('f_treat_typ_') or col.startswith('f_treat_24_meth_') or col.startswith('pre_tr_frt_kg_'):
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 1: Pre-sowing Treatment Calculation
for i in range(1, 7):
    df[f'tot_f_pre_treat_typ_{i}'] = df[[col for col in df.columns if col.startswith(f'f_pre_treat_typ_{i}_')]].sum(axis=1, skipna=True)
    df[f'tot_f_pre_treat_typ_24_{i}'] = df[[col for col in df.columns if col.startswith(f'f_pre_treat_typ_24_{i}_')]].sum(axis=1, skipna=True)

    df[f'f_pre_treat_1p_{i}'] = (df[f'tot_f_pre_treat_typ_{i}'] > 0).astype(int)
    df[f'f_pre_treat_1p_24_{i}'] = (df[f'tot_f_pre_treat_typ_24_{i}'] > 0).astype(int)

# Step 2: Total Amount of Fertilizer Applied Calculation
df['fert_total'] = df[[col for col in df.columns if col.startswith('pre_tr_frt_kg_')]].sum(axis=1, skipna=True)

# Step 3: Post-sowing Treatment Calculation
for i in range(1, 6):
    df[f'treat_typ_tot_{i}'] = df[[col for col in df.columns if col.startswith(f'f_treat_typ_{i}_')]].sum(axis=1, skipna=True)
    df[f'treat_typ_tot_24_{i}'] = df[[col for col in df.columns if col.startswith(f'f_treat_24_meth_{i}_')]].sum(axis=1, skipna=True)

    df[f'treat_typ_{i}'] = (df[f'treat_typ_tot_{i}'] > 0).astype(int)
    df[f'treat_typ_24_{i}'] = (df[f'treat_typ_tot_24_{i}'] > 0).astype(int)

# Additional indicators
df['fym_pre_23'] = ((df['f_pre_treat_1p_3'] == 1) | (df['treat_typ_2'] == 1)).astype(int)
df['fym_pre_24'] = ((df['f_pre_treat_1p_24_3'] == 1) | (df['treat_typ_24_2'] == 1)).astype(int)

df['compost_23'] = ((df['f_pre_treat_1p_4'] == 1) | (df['treat_typ_3'] == 1)).astype(int)
df['compost_24'] = ((df['f_pre_treat_1p_24_4'] == 1) | (df['treat_typ_24_3'] == 1)).astype(int)

df['Jivamrita_23'] = ((df['f_pre_treat_1p_5'] == 1) | (df['treat_typ_4'] == 1)).astype(int)
df['Jivamrita_24'] = ((df['f_pre_treat_1p_24_5'] == 1) | (df['treat_typ_24_4'] == 1)).astype(int)

df['green_man_23'] = ((df['f_pre_treat_1p_6'] == 1) | (df['treat_typ_5'] == 1)).astype(int)
df['green_man_24'] = ((df['f_pre_treat_1p_24_6'] == 1) | (df['treat_typ_24_5'] == 1)).astype(int)

df['no_chem_fert_23'] = ((df['f_pre_treat_1p_1'] == 0) & (df['treat_typ_1'] == 0)).astype(int)
df['no_chem_fert_24'] = ((df['f_pre_treat_1p_24_1'] == 0) & (df['treat_typ_24_1'] == 0)).astype(int)

df['chem_fert_23'] = ((df['f_pre_treat_1p_1'] == 1) | (df['treat_typ_1'] == 1)).astype(int)
df['chem_fert_24'] = ((df['f_pre_treat_1p_24_1'] == 1) | (df['treat_typ_24_1'] == 1)).astype(int)

# Optionally, display the DataFrame to verify changes
df[['fym_pre_23', 'compost_23', 'Jivamrita_23', 'green_man_23', 'no_chem_fert_23', 'chem_fert_23',
          'fym_pre_24', 'compost_24', 'Jivamrita_24', 'green_man_24', 'no_chem_fert_24', 'chem_fert_24']].head()


,fym_pre_23,compost_23,Jivamrita_23,green_man_23,no_chem_fert_23,chem_fert_23,fym_pre_24,compost_24,Jivamrita_24,green_man_24,no_chem_fert_24,chem_fert_24
0,0,0,0,0,1,0,0,0,0,0,1,0
1,0,0,0,0,1,0,0,0,0,0,1,0
2,0,0,0,0,1,0,1,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,0,0,0,1


### Manure Application Frequency


In [54]:

# Ensure all relevant columns are numeric
manure_columns = [f'f_treat_man_frq_{i}' for i in range(1, 7)]

# Check if columns exist
for col in manure_columns:
    if col not in df.columns:
        print(f"Column: {col}, Exists: False")
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate Manure application frequency
df['man_ap_freq'] = df[manure_columns].max(axis=1, skipna=True)

# Optionally convert to category and set labels
df['man_ap_freq'] = df['man_ap_freq'].astype('category')
df['man_ap_freq'] = df['man_ap_freq'].cat.rename_categories({
    1: "Once",
    2: "Twice",
    3: "Thrice",
    4: "Four times",
    5: "Five times",
    6: "Six times"
})

df['man_ap_freq'] = df['man_ap_freq'].cat.set_categories([
    "Once", 
    "Twice", 
    "Thrice", 
    "Four times", 
    "Five times", 
    "Six times"
], ordered=True)

# Display the DataFrame to verify changes
print(df[['man_ap_freq']].head())


  man_ap_freq
0         NaN
1         NaN
2         NaN
3         NaN
4         NaN


### Jivamrita/Jeevamrutham Application Frequency


In [55]:
# Ensure all relevant columns are numeric
jivamrita_columns = [f'f_treat_j_ap_frq_{i}' for i in range(1, 7)]

# Check if columns exist and convert to numeric
for col in jivamrita_columns:
    if col not in df.columns:
        print(f"Column: {col}, Exists: False")
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate Jivamrita/jeevamrutham application frequency
df['j_ap_freq'] = df[jivamrita_columns].max(axis=1, skipna=True)

# Optionally convert to category and set labels
df['j_ap_freq'] = df['j_ap_freq'].astype('category')
df['j_ap_freq'] = df['j_ap_freq'].cat.rename_categories({
    1: "Once",
    2: "Twice",
    3: "Thrice",
    4: "Four times",
    5: "Five times",
    6: "Six times"
})

df['j_ap_freq'] = df['j_ap_freq'].cat.set_categories([
    "Once", 
    "Twice", 
    "Thrice", 
    "Four times", 
    "Five times", 
    "Six times"
], ordered=True)

# Display the DataFrame to verify changes
print(df[['j_ap_freq']].head())


  j_ap_freq
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN


### Application Method

In [56]:
# Step 1: Ensure all relevant columns are numeric
for i in range(1, 5):
    treat_ap_meth_cols = [col for col in df.columns if col.startswith(f'f_treat_ap_meth_{i}_')]
    for col in treat_ap_meth_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Step 2: Calculate row totals for each application method
for i in range(1, 5):
    treat_ap_meth_cols = [col for col in df.columns if col.startswith(f'f_treat_ap_meth_{i}_')]
    df[f'treat_ap_meth_tot_{i}'] = df[treat_ap_meth_cols].sum(axis=1, skipna=True)

# Step 3: Generate binary indicators based on row totals
for i in range(1, 5):
    df[f'treat_ap_meth_{i}'] = 0
    df.loc[df[f'treat_ap_meth_tot_{i}'] > 0, f'treat_ap_meth_{i}'] = 1

# Display the DataFrame to verify changes
df[[f'treat_ap_meth_{i}' for i in range(1, 5)]].head()


,treat_ap_meth_1,treat_ap_meth_2,treat_ap_meth_3,treat_ap_meth_4
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


##  Mulch or soil cover before the start of 2023 Kharif 

In [57]:
# Ensure all relevant columns are numeric
mulch_cols = [f'f_mulch_{i}' for i in range(1, 7)]
for col in mulch_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate row max for mulch columns
df['f_mulch_1p'] = df[mulch_cols].max(axis=1, skipna=True)

# Display the DataFrame to verify changes
print(df['f_mulch_1p'].head())

# Define cover methods labels (they are not directly used in the calculation, so no need to store them in the DataFrame)
cover_methods_labels = {
    1: "mulch, e.g. made from crop residues",
    2: "cover crop",
    3: "trees or shrubs",
    4: "other",
    999: "Unsure/don't know"
}

# Process cover method columns
for i in range(1, 5):
    # Ensure all relevant columns are numeric
    cover_cols = [col for col in df.columns if col.startswith(f'f_cover_ls_meth_{i}_')]
    cover_cols_24 = [col for col in df.columns if col.startswith(f'f_cover_meth_24_{i}_')]
    for col in cover_cols + cover_cols_24:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Calculate row totals for each cover method
    df[f'cover_ls_meth_tot_{i}'] = df[cover_cols].sum(axis=1, skipna=True)
    df[f'cover_ls_meth_tot_24_{i}'] = df[cover_cols_24].sum(axis=1, skipna=True)

    # Generate binary indicators based on row totals
    df[f'cover_ls_meth_{i}'] = (df[f'cover_ls_meth_tot_{i}'] > 0).astype(int)
    df[f'cover_ls_meth_24_{i}'] = (df[f'cover_ls_meth_tot_24_{i}'] > 0).astype(int)

    # Display the DataFrame to verify changes
    print(df[[f'cover_ls_meth_{i}', f'cover_ls_meth_24_{i}']].head())

# Generate binary indicators for mulch and cover crop/tree columns
df['mulch_23'] = df['cover_ls_meth_1']
df['mulch_24'] = df['cover_ls_meth_24_1']
df['cover_crop_23'] = df['cover_ls_meth_2']
df['cover_crop_24'] = df['cover_ls_meth_24_2']
df['cover_tree_23'] = df['cover_ls_meth_3']
df['cover_tree_24'] = df['cover_ls_meth_24_3']

# Display the DataFrame to verify changes
df[['mulch_23', 'mulch_24', 'cover_crop_23', 'cover_crop_24', 'cover_tree_23', 'cover_tree_24']].head()


0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: f_mulch_1p, dtype: float64
   cover_ls_meth_1  cover_ls_meth_24_1
0                0                   0
1                0                   0
2                0                   0
3                0                   0
4                0                   0
   cover_ls_meth_2  cover_ls_meth_24_2
0                0                   0
1                0                   0
2                0                   0
3                0                   0
4                0                   0
   cover_ls_meth_3  cover_ls_meth_24_3
0                0                   0
1                0                   0
2                0                   0
3                0                   0
4                0                   0
   cover_ls_meth_4  cover_ls_meth_24_4
0                0                   0
1                0                   0
2                0                   0
3                0                   0
4                0            

,mulch_23,mulch_24,cover_crop_23,cover_crop_24,cover_tree_23,cover_tree_24
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


## Irrigation 

In [58]:

# Ensure all relevant columns are numeric
irrigate_cols = [f'f_irrigate_{i}' for i in range(1, 7)]
for col in irrigate_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Generate the 'f_irrigate' column
df['f_irrigate'] = 0
for col in irrigate_cols:
    df['f_irrigate'] = df['f_irrigate'].where(df[col] != 1, 1)

# Define irrigation type labels (they are not directly used in the calculation, so no need to store them in the DataFrame)
irrigation_types = {
    1: "drip",
    2: "sprinkler",
    3: "furrow",
    4: "flood"
}

# Ensure irrigation type columns are numeric
for i in range(1, 5):
    for j in range(1, 7):
        col = f'f_irrigate_type_{i}_{j}'
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate row totals for each irrigation type
for i in range(1, 5):
    irrigate_type_cols = [col for col in df.columns if col.startswith(f'f_irrigate_type_{i}_')]
    irrigate_type_cols_24 = [col for col in df.columns if col.startswith(f'f_irrigate_24_type_{i}_')]
    
    df[f'irrigate_type_tot_{i}'] = df[irrigate_type_cols].sum(axis=1, skipna=True)
    df[f'irrigate_type_tot_24_{i}'] = df[irrigate_type_cols_24].sum(axis=1, skipna=True)

    # Generate binary indicators based on row totals
    df[f'irrigation_typ_{i}'] = (df[f'irrigate_type_tot_{i}'] > 0).astype(int)
    df[f'irrigation_typ_24_{i}'] = (df[f'irrigate_type_tot_24_{i}'] > 0).astype(int)

    # Display the DataFrame to verify changes
    print(df[[f'irrigation_typ_{i}', f'irrigation_typ_24_{i}']].head())

# Generate binary indicators for irrigation types
df['drip_irr_23'] = df['irrigation_typ_1']
df['drip_irr_24'] = df['irrigation_typ_24_1']
df['sprink_irr_23'] = df['irrigation_typ_2']
df['sprink_irr_24'] = df['irrigation_typ_24_2']
df['furrow_irr_23'] = df['irrigation_typ_3']
df['furrow_irr_24'] = df['irrigation_typ_24_3']
df['flood_irr_23'] = df['irrigation_typ_4']
df['flood_irr_24'] = df['irrigation_typ_24_4']

# Display the DataFrame to verify changes
df[['drip_irr_23', 'drip_irr_24', 'sprink_irr_23', 'sprink_irr_24', 'furrow_irr_23', 'furrow_irr_24', 'flood_irr_23', 'flood_irr_24']].head()


   irrigation_typ_1  irrigation_typ_24_1
0                 0                    0
1                 1                    0
2                 0                    0
3                 0                    0
4                 0                    0
   irrigation_typ_2  irrigation_typ_24_2
0                 0                    0
1                 0                    0
2                 1                    0
3                 0                    0
4                 0                    0
   irrigation_typ_3  irrigation_typ_24_3
0                 1                    0
1                 0                    0
2                 0                    0
3                 0                    0
4                 0                    0
   irrigation_typ_4  irrigation_typ_24_4
0                 0                    0
1                 1                    0
2                 1                    0
3                 1                    0
4                 1                    0


,drip_irr_23,drip_irr_24,sprink_irr_23,sprink_irr_24,furrow_irr_23,furrow_irr_24,flood_irr_23,flood_irr_24
0,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,1,0
2,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0


##  Energy for irrigation pump

In [59]:
import pandas as pd

# Ensure all relevant columns are numeric
pump_cols = [f'f_irrigate_pump_{i}' for i in range(1, 7)]
for col in pump_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Generate the 'irrigation_pump' column
df['irrigation_pump'] = df[pump_cols].max(axis=1)

# Calculate row totals for each pump type
for i in range(1, 6):
    pump_type_cols = [col for col in df.columns if col.startswith(f'irrigate_pump_type_{i}_')]
    pump_type_cols_24 = [col for col in df.columns if col.startswith(f'irrigate_pump_24_type_{i}_')]
    
    df[f'irrigate_pump_type_tot_{i}'] = df[pump_type_cols].sum(axis=1, skipna=True)
    df[f'irrigate_pump_type_tot_24_{i}'] = df[pump_type_cols_24].sum(axis=1, skipna=True)

    # Generate binary indicators based on row totals
    df[f'irrigation_pump_type_{i}'] = (df[f'irrigate_pump_type_tot_{i}'] > 0).astype(int)
    df[f'irrigation_pump_type_24_{i}'] = (df[f'irrigate_pump_type_tot_24_{i}'] > 0).astype(int)

# Define irrigation pump type labels (they are not directly used in the calculation, so no need to store them in the DataFrame)
pump_types = {
    1: "diesel",
    2: "gasoline",
    3: "manual",
    4: "electric (grid)",
    5: "solar"
}

# Generate binary indicators for irrigation pump types
df['solar_irri_23'] = df['irrigation_pump_type_5']
df['solar_irri_24'] = df['irrigation_pump_type_24_5']
df['diesel_pump_23'] = df['irrigation_pump_type_1']
df['diesel_pump_24'] = df['irrigation_pump_type_24_1']
df['gasoline_pump_23'] = df['irrigation_pump_type_2']
df['gasoline_pump_24'] = df['irrigation_pump_type_24_2']
df['manual_pump_23'] = df['irrigation_pump_type_3']
df['manual_pump_24'] = df['irrigation_pump_type_24_3']
df['elect_pump_23'] = df['irrigation_pump_type_4']
df['elect_pump_24'] = df['irrigation_pump_type_24_4']

# Display the DataFrame to verify changes
print(df[['solar_irri_23', 'solar_irri_24', 'diesel_pump_23', 'diesel_pump_24', 'gasoline_pump_23', 'gasoline_pump_24', 'manual_pump_23', 'manual_pump_24', 'elect_pump_23', 'elect_pump_24']].head())

# Calculate row totals for irrigation sources
for i in range(1, 6):
    source_cols = [col for col in df.columns if col.startswith(f'f_irr_source_{i}_')]
    df[f'irrg_source_tot_{i}'] = df[source_cols].sum(axis=1, skipna=True)

    # Generate binary indicators based on row totals
    df[f'irrg_source_{i}'] = (df[f'irrg_source_tot_{i}'] > 0).astype(int)

# Define irrigation source labels (they are not directly used in the calculation, so no need to store them in the DataFrame)
source_types = {
    1: "river",
    2: "lake",
    3: "pond",
    4: "well",
    5: "rainwater harvesting"
}

# Display the DataFrame to verify changes
df[[f'irrg_source_{i}' for i in range(1, 6)]].head()


   solar_irri_23  solar_irri_24  diesel_pump_23  diesel_pump_24  \
0              0              0               0               0   
1              0              0               0               0   
2              0              0               1               0   
3              0              0               0               0   
4              0              0               0               0   

   gasoline_pump_23  gasoline_pump_24  manual_pump_23  manual_pump_24  \
0                 0                 0               0               0   
1                 0                 0               0               0   
2                 0                 0               0               0   
3                 0                 0               0               0   
4                 0                 0               0               0   

   elect_pump_23  elect_pump_24  
0              0              0  
1              0              0  
2              0              0  
3              0      

,irrg_source_1,irrg_source_2,irrg_source_3,irrg_source_4,irrg_source_5
0,1,0,1,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


## Weed control

In [60]:

# Ensure all relevant columns are numeric
weed_sub_cols = [col for col in df.columns if col.startswith('f_weed_sub_')]
for col in weed_sub_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Generate the 'weed_sub' column
df['weed_sub'] = df[weed_sub_cols].max(axis=1)
df['weed_sub'] = df['weed_sub'].apply(lambda x: 1 if x > 0 else 0)
df['weed_sub'] = df['weed_sub'].astype('category')

# Rename categories
df['weed_sub'] = df['weed_sub'].cat.rename_categories({1: "Yes", 0: "No"})
df['weed_sub'] = df['weed_sub'].cat.set_categories(["Yes", "No"], ordered=True)

# Define herbicide labels (they are not directly used in the calculation, so no need to store them in the DataFrame)
herbicide_labels = {
    1: "Chemical Herbicide",
    2: "Natural Herbicide",
    3: "Other"
}

# Weed control substance type
herbcides_cols = []
herbcides_24_cols = []
for i in range(1, 4):
    herbicide_cols = [col for col in df.columns if col.startswith(f'herbicide_{i}_')]
    herbicide_cols_24 = [col for col in df.columns if col.startswith(f'f_weed_con_meth_24_{i}_')]
    
    df[f'herbcides_{i}'] = df[herbicide_cols].max(axis=1)
    df[f'herbcides_24_{i}'] = df[herbicide_cols_24].max(axis=1)
    
    df[f'herbcides_{i}'] = df[f'herbcides_{i}'].fillna(0).astype(int)
    df[f'herbcides_24_{i}'] = df[f'herbcides_24_{i}'].fillna(0).astype(int)
    
    herbcides_cols.append(f'herbcides_{i}')
    herbcides_24_cols.append(f'herbcides_24_{i}')

# Generate binary indicators for herbicides
herbicide_indicators = {
    'nat_herb_23': df['herbcides_2'],
    'nat_herb_24': df['herbcides_24_2'],
    'no_chem_herb_23': (df['herbcides_1'] == 0).astype(int),
    'no_chem_herb_24': (df['herbcides_24_1'] == 0).astype(int),
    'chem_herb_23': df['herbcides_1'],
    'chem_herb_24': df['herbcides_24_1']
}

# Add columns using pd.concat
df = pd.concat([df, pd.DataFrame(herbicide_indicators)], axis=1)

# Optional: add labels to the columns
column_labels = {
    'nat_herb_23': "Natural Herb",
    'nat_herb_24': "Natural Herb",
    'no_chem_herb_23': "No C Herb",
    'no_chem_herb_24': "No C Herb",
    'chem_herb_23': "Chemical Herb.",
    'chem_herb_24': "Chemical Herb."
}

# Display the DataFrame to verify changes
df[['weed_sub', 'nat_herb_23', 'nat_herb_24', 'no_chem_herb_23', 'no_chem_herb_24', 'chem_herb_23', 'chem_herb_24']].head()


,weed_sub,nat_herb_23,nat_herb_24,no_chem_herb_23,no_chem_herb_24,chem_herb_23,chem_herb_24
0,No,0,0,1,1,0,0
1,No,0,0,1,1,0,0
2,No,0,0,1,1,0,0
3,Yes,0,0,0,1,1,0
4,No,0,0,1,1,0,0


## Pest and disease control substance 

In [61]:

# Convert relevant columns to numeric, forcing errors to NaN
pest_control_cols = [f'f_pest_con_{x}' for x in range(1, 7)]
for col in pest_control_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').replace(999, pd.NA)

# Generate the 'pest_control' column
df['pest_control'] = df[pest_control_cols].max(axis=1, skipna=True)
df['pest_control'] = df['pest_control'].apply(lambda x: 1 if pd.notna(x) else 0)
df['pest_control'] = df['pest_control'].astype('category')
df['pest_control'] = df['pest_control'].cat.rename_categories({1: "Yes", 0: "No"})

# Define pest control method labels (not directly used in calculation, so no need to store them in DataFrame)
pest_con_typ_labels = {
    1: "Chemical Pest Control",
    2: "Biological Pest Control",
    3: "Integrated Pest Management"
}

# Type of pest control method used
for i in range(1, 4):
    pest_control_type_cols = [col for col in df.columns if col.startswith(f'f_pest_con_typ_{i}_')]
    pest_control_24_type_cols = [col for col in df.columns if col.startswith(f'f_pest_con_meth_24_{i}_')]

    # Ensure the columns are numeric
    df[pest_control_type_cols] = df[pest_control_type_cols].apply(pd.to_numeric, errors='coerce')
    df[pest_control_24_type_cols] = df[pest_control_24_type_cols].apply(pd.to_numeric, errors='coerce')

    df[f'pest_con_tot_{i}'] = df[pest_control_type_cols].sum(axis=1, skipna=True)
    df[f'pest_con_tot_24_{i}'] = df[pest_control_24_type_cols].sum(axis=1, skipna=True)

    df[f'pest_con_type_{i}'] = (df[f'pest_con_tot_{i}'] > 0).astype(int)
    df[f'pest_con_type_24_{i}'] = (df[f'pest_con_tot_24_{i}'] > 0).astype(int)

# Display the DataFrame to verify changes
#import ace_tools as tools; tools.display_dataframe_to_user(name="Pest Control DataFrame", dataframe=df)

print(df[['pest_control'] + [f'pest_con_type_{i}' for i in range(1, 4)] + [f'pest_con_type_24_{i}' for i in range(1, 4)]].head())


  pest_control  pest_con_type_1  pest_con_type_2  pest_con_type_3  \
0          Yes                1                0                0   
1          Yes                1                0                0   
2          Yes                0                0                0   
3          Yes                1                0                0   
4          Yes                0                0                0   

   pest_con_type_24_1  pest_con_type_24_2  pest_con_type_24_3  
0                   0                   0                   0  
1                   0                   0                   0  
2                   0                   0                   0  
3                   0                   0                   0  
4                   0                   0                   0  


##  Application method 

In [62]:


# Function to ensure numeric conversion
def convert_to_numeric(df, cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Chemical pesticides application frequency
pest_ap_frq_cols = [col for col in df.columns if col.startswith('f_chem_ap_fq_')]
df = convert_to_numeric(df, pest_ap_frq_cols)
df['pest_ap_frq'] = df[pest_ap_frq_cols].max(axis=1, skipna=True)
df['pest_ap_frq'] = df['pest_ap_frq'].replace({5: 4, 6: 4})
df['pest_ap_frq'] = df['pest_ap_frq'].astype('category')
df['pest_ap_frq'] = df['pest_ap_frq'].cat.rename_categories({
    1: "1 time only", 2: "2-3 times", 3: "4-5 times", 4: "over 6 times"
})

# Natural pesticides application frequency
f_nat_ap_frq_cols = [col for col in df.columns if col.startswith('f_nat_ap_fq_')]
df = convert_to_numeric(df, f_nat_ap_frq_cols)
df['f_nat_ap_frq'] = df[f_nat_ap_frq_cols].max(axis=1, skipna=True)
df['f_nat_ap_frq'] = df['f_nat_ap_frq'].replace({5: 4, 6: 4})
df['f_nat_ap_frq'] = df['f_nat_ap_frq'].astype('category')
df['f_nat_ap_frq'] = df['f_nat_ap_frq'].cat.rename_categories({
    1: "1 time only", 2: "2-3 times", 3: "4-5 times", 4: "over 6 times"
})

# Application of growth stimulant
treat_grwth_stim_cols = [col for col in df.columns if col.startswith('f_treat_')]
df = convert_to_numeric(df, treat_grwth_stim_cols)
df['treat_grwth_stim'] = df[treat_grwth_stim_cols].max(axis=1, skipna=True)
df['treat_grwth_stim'] = df['treat_grwth_stim'].apply(lambda x: 1 if x > 0 else 0)
df['treat_grwth_stim'] = df['treat_grwth_stim'].astype('category')
df['treat_grwth_stim'] = df['treat_grwth_stim'].cat.rename_categories({
    1: "Yes", 0: "No"
})

# Crop labels (assuming `crop_var_` columns exist and should be labeled)
crop_labels = {
    1: "Paddy rice", 2: "Cotton", 3: "Chillies", 4: "Maize", 5: "Redgram",
    6: "Greengram", 7: "Blackgram", 8: "Common bean", 11: "Groundnut",
    12: "Soybean", 13: "Jowar (Sorghum)", 14: "Bajra (Pearl Millet)",
    17: "Castor", 19: "Sessamun", 20: "Sugarcane", 21: "Wheat",
    22: "Mustard", 23: "Banana", 32: "Guava", 40: "Other vegetables",
    44: "Other", 45: "Cumin seed", 46: "Dungli", 47: "Ajmo", 48: "Kontola"
}

for i in crop_labels.keys():
    col_name = f'crop_var_{i}'
    if col_name in df.columns:
        df[col_name] = df[col_name].astype('category')
        df[col_name].cat.rename_categories({1: crop_labels[i]}, inplace=True)

# Display the DataFrame to verify changes
#import ace_tools as tools; tools.display_dataframe_to_user(name="Updated DataFrame", dataframe=df)

# Printing a subset of columns to check changes
df[['pest_ap_frq', 'f_nat_ap_frq', 'treat_grwth_stim']].head()


,pest_ap_frq,f_nat_ap_frq,treat_grwth_stim
0,4-5 times,NaN,Yes
1,over 6 times,NaN,Yes
2,NaN,NaN,Yes
3,over 6 times,NaN,Yes
4,NaN,NaN,Yes


## seed treatment

In [63]:

# Initialize new columns
df['sd_not_treated'] = 0
df['sd_treated_purchased'] = 0
df['sd_hh_treated'] = 0

# Replace values based on conditions
df['sd_not_treated'] = df[['f_yn_sd_tr_1', 'f_yn_sd_tr_2', 'f_yn_sd_tr_3']].apply(lambda x: 1 if any(x == 0) else 0, axis=1)
df['sd_treated_purchased'] = df[['f_yn_sd_tr_1', 'f_yn_sd_tr_2', 'f_yn_sd_tr_3']].apply(lambda x: 1 if any(x == 2) else 0, axis=1)
df['sd_hh_treated'] = df[['f_yn_sd_tr_1', 'f_yn_sd_tr_2', 'f_yn_sd_tr_3']].apply(lambda x: 1 if any(x == 1) else 0, axis=1)

# Labels are just for documentation in this case
labels = {
    'sd_not_treated': "HH did not treat seed before sowing",
    'sd_treated_purchased': "HH used already treated seed when purchased",
    'sd_hh_treated': "HH did something to treat the seed"
}



## treatment used - was not asked for 2020 recall

In [64]:
# Calculate row totals for seed treatment methods
for i in range(1, 7):
    col_tot = f'sd_trt_list_tot_{i}'
    col_method = f'sd_trt_methd_{i}'
    treatment_cols = [col for col in df.columns if col.startswith(f'sd_trt_list_{i}_')]
    if treatment_cols:  # Only proceed if there are matching columns
        df[col_tot] = df[treatment_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1, skipna=True)
        df[col_method] = (df[col_tot] > 0).astype(int)

# Display the DataFrame to verify changes
df.head()

,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,sd_trt_list_tot_2,sd_trt_methd_2,sd_trt_list_tot_3,sd_trt_methd_3,sd_trt_list_tot_4,sd_trt_methd_4,sd_trt_list_tot_5,sd_trt_methd_5,sd_trt_list_tot_6,sd_trt_methd_6
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


## Seed sowing method

In [65]:

# Ensure the required columns are present in the DataFrame
required_columns = [
    'sd_trt_list_1', 'sd_trt_list_2', 'sd_trt_list_3', 'sd_trt_list_4', 'sd_trt_list_5', 'sd_trt_list_6',
    'sd_trt_methd_1', 'sd_trt_methd_2', 'sd_trt_methd_3', 'sd_trt_methd_4', 'sd_trt_methd_5', 'sd_trt_methd_6',
    'pest_con_type_1', 'pest_con_type_2', 'pest_con_type_24_2', 'pest_con_type_24_1',
    'j_liq_sol_2_1', 'j_liq_sol_2_2', 'j_liq_sol_2_3', 'j_liq_sol_2_4', 'j_liq_sol_2_5', 'j_liq_sol_2_6',
    'j_liq_sol_1_1', 'j_liq_sol_1_2', 'j_liq_sol_1_3', 'j_liq_sol_1_4', 'j_liq_sol_1_5', 'j_liq_sol_1_6',
    'Jivamrita_23'
]

# Create missing columns with default values if they do not exist
for col in required_columns:
    if col not in df.columns:
        df[col] = 0

# Convert all required columns to numeric type
for col in required_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Treatment used - was not asked for 2020 recall
for i in range(1, 7):
    col_tot = f'sd_trt_list_tot_{i}'
    col_method = f'sd_trt_methd_{i}'
    treatment_cols = [col for col in df.columns if col.startswith(f'sd_trt_list_{i}_')]
    
    # Ensure all treatment columns are numeric
    for col in treatment_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    if treatment_cols:  # Ensure there are columns to sum
        df[col_tot] = df[treatment_cols].sum(axis=1, skipna=True)
        df[col_method] = (df[col_tot] > 0).astype(int)

# Initialize columns
df['sd_trt_chemical'] = 0
df['sd_trt_organic'] = 0
df['beejam_23'] = 0

# Replace based on conditions
df['sd_trt_chemical'] = ((df['sd_trt_methd_1'] == 1) | (df['sd_trt_methd_2'] == 1)).astype(int)
df['sd_trt_organic'] = ((df['sd_trt_methd_3'] == 1) | (df['sd_trt_methd_5'] == 1)).astype(int)
df['beejam_23'] = df['sd_trt_methd_4']

# Generate columns based on conditions
df['botan_23'] = ((df['pest_con_type_2'] == 1) | (df['sd_trt_organic'] == 1)).astype(int)
df['botan_24'] = (df['pest_con_type_24_2'] == 1).astype(int)

df['no_chem_pest_23'] = ((df['pest_con_type_1'] == 0) & (df['sd_trt_chemical'] == 0)).astype(int)
df['no_chem_pest_24'] = (df['pest_con_type_24_1'] == 0).astype(int)

df['chem_pest_23'] = ((df['pest_con_type_1'] == 1) | (df['sd_trt_chemical'] == 1)).astype(int)
df['chem_pest_24'] = (df['pest_con_type_24_1'] == 1).astype(int)

# Row max calculations
df['ghana_23'] = df[[f'j_liq_sol_2_{i}' for i in range(1, 7)]].max(axis=1)
df['drava_23'] = df[[f'j_liq_sol_1_{i}' for i in range(1, 7)]].max(axis=1)
df['ghana_23'] = df['ghana_23'].fillna(0).astype(int)
df['drava_23'] = df['drava_23'].fillna(0).astype(int)
df['Jivamrita_23'] = df.apply(lambda row: 1 if row['Jivamrita_23'] == 0 and (row['ghana_23'] == 1 or row['drava_23'] == 1) else row['Jivamrita_23'], axis=1)



## Crop residue management

In [66]:


# Initialize necessary columns for residue treatment management
for i in range(1, 12):
    for j in range(1, 7):
        col_new = f'residue_treat_24_dif_{i}_{j}'
        col_old = f'residue_treat_{i}_{j}'
        col_cond = f'residue_treat_24_{j}'
        if col_old in df.columns and col_cond in df.columns:
            df[col_new] = df[col_old].where(df[col_cond] == 1, df.get(col_new, None))

# Ensure all relevant columns are numeric before summation
for i in range(1, 12):
    for j in range(1, 7):
        col = f'residue_treat_{i}_{j}'
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        col = f'residue_treat_24_dif_{i}_{j}'
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate row totals for residue treatment
for i in range(1, 12):
    cols = [f'residue_treat_{i}_{j}' for j in range(1, 7) if f'residue_treat_{i}_{j}' in df.columns]
    df[f'residue_treat_tot_{i}'] = df[cols].sum(axis=1, skipna=True)

    cols_24 = [f'residue_treat_24_dif_{i}_{j}' for j in range(1, 7) if f'residue_treat_24_dif_{i}_{j}' in df.columns]
    df[f'residue_treat_tot_24_{i}'] = df[cols_24].sum(axis=1, skipna=True)

    # Initialize new columns for residue treatment
    df[f'residue_treatment_{i}'] = 0
    df[f'residue_treatment_24_{i}'] = 0

    # Conditional replacements
    df.loc[(df[f'residue_treat_tot_{i}'] > 0) & (df[f'residue_treat_tot_{i}'].notna()), f'residue_treatment_{i}'] = 1
    df.loc[(df[f'residue_treat_tot_24_{i}'] > 0) & (df[f'residue_treat_tot_24_{i}'].notna()), f'residue_treatment_24_{i}'] = 1

    # Label the new columns (just as a comment because labels are not used in Python like in Stata)
    # lab var residue_treatment_{i} "${residue_treat_{i}}"
    # lab var residue_treatment_24_{i} "${residue_treat_{i}}"

# Display the updated DataFrame to verify the changes
df.head()


,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,residue_treatment_9,residue_treatment_24_9,residue_treat_tot_10,residue_treat_tot_24_10,residue_treatment_10,residue_treatment_24_10,residue_treat_tot_11,residue_treat_tot_24_11,residue_treatment_11,residue_treatment_24_11
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0.0,0.0,0,0
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,0,0,0.0,0.0,0,0,0.0,0.0,0,0
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0.0,0.0,0,0
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,0,0,0.0,0.0,0,0,0.0,0.0,0,0
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0,0,0.0,0.0,0,0,0.0,0.0,0,0


In [67]:

# Initialize necessary columns for residue treatment management
for i in range(1, 12):
    for j in range(1, 7):
        col_new = f'residue_treat_24_dif_{i}_{j}'
        col_old = f'residue_treat_{i}_{j}'
        col_cond = f'residue_treat_24_{j}'
        if col_old in df.columns and col_cond in df.columns:
            df[col_new] = df[col_old].where(df[col_cond] == 1, df.get(col_new, None))

# Ensure all relevant columns are numeric before summation
for i in range(1, 12):
    for j in range(1, 7):
        col = f'residue_treat_{i}_{j}'
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        col = f'residue_treat_24_dif_{i}_{j}'
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

# Calculate row totals for residue treatment
for i in range(1, 12):
    cols = [f'residue_treat_{i}_{j}' for j in range(1, 7) if f'residue_treat_{i}_{j}' in df.columns]
    df[f'residue_treat_tot_{i}'] = df[cols].sum(axis=1, skipna=True)

    cols_24 = [f'residue_treat_24_dif_{i}_{j}' for j in range(1, 7) if f'residue_treat_24_dif_{i}_{j}' in df.columns]
    df[f'residue_treat_tot_24_{i}'] = df[cols_24].sum(axis=1, skipna=True)

    # Initialize new columns for residue treatment
    df[f'residue_treatment_{i}'] = 0
    df[f'residue_treatment_24_{i}'] = 0

    # Conditional replacements
    df.loc[(df[f'residue_treat_tot_{i}'] > 0) & (df[f'residue_treat_tot_{i}'].notna()), f'residue_treatment_{i}'] = 1
    df.loc[(df[f'residue_treat_tot_24_{i}'] > 0) & (df[f'residue_treat_tot_24_{i}'].notna()), f'residue_treatment_24_{i}'] = 1

    # Label the new columns (just as a comment because labels are not used in Python like in Stata)
    # lab var residue_treatment_{i} "${residue_treat_{i}}"
    # lab var residue_treatment_24_{i} "${residue_treat_{i}}"

# Generate residue indicators for 2021 and 2020
df['residue_23'] = ((df['residue_treatment_6'] == 1) | (df['residue_treatment_8'] == 1) | (df['residue_treatment_10'] == 1)).astype(int)
df['residue_24'] = ((df['residue_treatment_24_6'] == 1) | (df['residue_treatment_24_8'] == 1) | (df['residue_treatment_24_10'] == 1)).astype(int)

# Label the new columns (just as a comment because labels are not used in Python like in Stata)
# lab var residue_21 "Crop Residue"
# lab var residue_20 "Crop Residue"

# Destring and replace f_tree_2020_dif_* columns
for i in range(1, 7):
    col = f'f_tree_2024_dif_{i}'
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Display the updated DataFrame to verify the changes
df.head()


,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,residue_treat_tot_10,residue_treat_tot_24_10,residue_treatment_10,residue_treatment_24_10,residue_treat_tot_11,residue_treat_tot_24_11,residue_treatment_11,residue_treatment_24_11,residue_23,residue_24
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0


In [68]:


# Ensure all relevant columns are numeric before performing operations
for i in range(1, 7):
    col = f'f_tree_2024_dif_{i}'
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    col = f'tree_chg_3_{i}'
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Conditional replacements for f_plant_pat_tree_20_2_* columns
for i in range(1, 7):
    col_new = f'f_plant_pat_tree_24_2_{i}'
    col_old = f'f_plant_pat_tree_2_{i}'
    col_cond1 = f'f_tree_2024_dif_{i}'
    col_cond2 = f'tree_chg_3_{i}'
    if col_old in df.columns and col_cond1 in df.columns and col_cond2 in df.columns:
        df[col_new] = df.apply(lambda row: row[col_old] if row[col_cond1] == 0 and row[col_cond2] != 1 else row[col_new], axis=1)

# Display the updated DataFrame to verify the changes
df.head()


,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,residue_treat_tot_10,residue_treat_tot_24_10,residue_treatment_10,residue_treatment_24_10,residue_treat_tot_11,residue_treat_tot_24_11,residue_treatment_11,residue_treatment_24_11,residue_23,residue_24
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0.0,0.0,0,0,0,0


In [69]:

# Generate tree_bound_23 and tree_bound_24 columns based on the conditions
tree_bound_23_cols = [f'f_plant_pat_tree_2_{i}' for i in range(1, 7) if f'f_plant_pat_tree_2_{i}' in df.columns]
tree_bound_24_cols = [f'f_plant_pat_tree_24_2_{i}' for i in range(1, 7) if f'f_plant_pat_tree_24_2_{i}' in df.columns]

df['tree_bound_23'] = df[tree_bound_23_cols].eq(1).any(axis=1).astype(int)
df['tree_bound_24'] = df[tree_bound_24_cols].eq(1).any(axis=1).astype(int)

# Label the new columns (just as a comment because labels are not used in Python like in Stata)
# lab var tree_bound_23 "Boundary tree"
# lab var tree_bound_24 "Boundary tree"

# Display the updated DataFrame to verify the changes
print(df[['tree_bound_23', 'tree_bound_24']].head())


   tree_bound_23  tree_bound_24
0              0              0
1              0              0
2              1              0
3              0              0
4              1              0


In [70]:


# Initialize soil cover columns
df['soil_cov_pre_23'] = 0
df['soil_cov_pre_24'] = 0

# Ensure relevant columns are numeric and replace soil cover based on conditions
for i in range(1, 7):
    col_23 = f'f_cover_ls_per_{i}'
    col_24 = f'f_cover_per_24_{i}'
    
    # Convert columns to numeric
    if col_23 in df.columns:
        df[col_23] = pd.to_numeric(df[col_23], errors='coerce')
    if col_24 in df.columns:
        df[col_24] = pd.to_numeric(df[col_24], errors='coerce')
    
    # Apply conditional replacements
    if col_23 in df.columns:
        df.loc[(df[col_23] > 2) & (df[col_23].notna()), 'soil_cov_pre_23'] = 1
    if col_24 in df.columns:
        df.loc[(df[col_24] > 2) & (df[col_24].notna()), 'soil_cov_pre_24'] = 1

# Label the new columns (just as a comment because labels are not used in Python like in Stata)
# lab var soil_cov_pre_23 "Pre-crop Cover"
# lab var soil_cov_pre_24 "Pre-crop Cover"

# Display the updated DataFrame to verify the changes
print(df[['soil_cov_pre_23', 'soil_cov_pre_24']].head())


   soil_cov_pre_23  soil_cov_pre_24
0                0                0
1                0                0
2                0                0
3                0                0
4                0                0


##  Intercropping

In [71]:


# Loop through the columns and perform the conditional replacement
for i in range(1, 7):
    col_new = f'f_crp_arg_20_dif_how_{i}'
    col_old = f'f_crp_arg_{i}'
    col_cond = f'f_crp_arg_20_same_{i}'

    # Convert columns to numeric to ensure they can be processed
    if col_new in df.columns:
        df[col_new] = pd.to_numeric(df[col_new], errors='coerce')
    if col_cond in df.columns:
        df[col_cond] = pd.to_numeric(df[col_cond], errors='coerce')

    # Apply conditional replacement
    if col_new in df.columns and col_old in df.columns and col_cond in df.columns:
        df[col_new] = df.apply(lambda row: row[col_old] if row[col_cond] == 1 else row[col_new], axis=1)

# Display the updated DataFrame to verify the changes
df.head()


,deviceid,device_num,text_audit,state_name,dis_name,block,village_name,audio_audit_1,audio_audit_2,audio_audit_3,...,residue_treat_tot_11,residue_treat_tot_24_11,residue_treatment_11,residue_treatment_24_11,residue_23,residue_24,tree_bound_23,tree_bound_24,soil_cov_pre_23,soil_cov_pre_24
0,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0,0,0,0,0
1,3ebda66e2e178c42,9.512526e+09,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,NaN,...,0.0,0.0,0,0,0,0,0,0,0,0
2,26a320911e7e2e98,9.193132e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0,1,0,0,0
3,77ba5160422fc308,9.195109e+11,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,https://icraf.surveycto.com/view/submission-at...,NaN,https://icraf.surveycto.com/view/submission-at...,...,0.0,0.0,0,0,0,0,0,0,0,0
4,3bc7b409f5c91501,NaN,https://icraf.surveycto.com/view/submission-at...,Gujarat,Devbhumi Dwarka,Bhatiya,Bhatavadiya,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0,1,0,0,0


In [72]:


# Generate inter_crop_21 column based on the conditions
df['inter_crop_23'] = (
    (df['f_crp_arg_1'] == 2) |
    (df['f_crp_arg_2'] == 2) |
    (df['f_crp_arg_3'] == 2) |
    (df['f_crp_arg_4'] == 2) |
    (df['f_crp_arg_5'] == 2) |
    (df['f_crp_arg_6'] == 2)
).astype(int)

# Initialize inter_crop_20 column
df['inter_crop_24'] = 0

# Conditional replacement for inter_crop_20 based on the specified conditions
for i in range(1, 7):
    col_same = f'f_crp_arg_20_same_{i}'
    if col_same in df.columns:
        df[col_same] = pd.to_numeric(df[col_same], errors='coerce')
        df['inter_crop_24'] = df.apply(lambda row: 1 if (
            (row['f_crp_arg_1'] == 2 or row['f_crp_arg_2'] == 2 or row['f_crp_arg_3'] == 2 or
             row['f_crp_arg_4'] == 2 or row['f_crp_arg_5'] == 2 or row['f_crp_arg_6'] == 2) and
            row[col_same] != 1) else row['inter_crop_24'], axis=1)

# Display the updated DataFrame to verify the changes
df[['inter_crop_23', 'inter_crop_24']].head()


,inter_crop_23,inter_crop_24
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [73]:


# List of columns to check for multi_24
crp_arg_columns_24 = [f'f_crp_arg_24_dif_how_{i}' for i in range(1, 7)]

# Check and ensure columns exist before using them for multi_24
for col in crp_arg_columns_24:
    if col not in df.columns:
        df[col] = 0  # or pd.NA or any appropriate default value

# Generate multi_23 column based on the conditions
df['multi_23'] = (
    (df['f_crp_arg_1'] == 6) |
    (df['f_crp_arg_2'] == 6) |
    (df['f_crp_arg_3'] == 6) |
    (df['f_crp_arg_4'] == 6) |
    (df['f_crp_arg_5'] == 6) |
    (df['f_crp_arg_6'] == 6)
).astype(int)

# Generate multi_24 column based on the conditions
df['multi_24'] = (
    (df['f_crp_arg_24_dif_how_1'] == 6) |
    (df['f_crp_arg_24_dif_how_2'] == 6) |
    (df['f_crp_arg_24_dif_how_3'] == 6) |
    (df['f_crp_arg_24_dif_how_4'] == 6) |
    (df['f_crp_arg_24_dif_how_5'] == 6) |
    (df['f_crp_arg_24_dif_how_6'] == 6)
).astype(int)

# Ensure 'inter_crop_23' and 'inter_crop_24' exist
if 'inter_crop_23' not in df.columns:
    df['inter_crop_23'] = 0  # or any appropriate default value

if 'inter_crop_24' not in df.columns:
    df['inter_crop_24'] = 0  # or any appropriate default value

# Replace inter_crop_23 based on the conditions
df['inter_crop_23'] = df.apply(lambda row: row['multi_23'] if row['inter_crop_23'] == 0 and row['multi_23'] == 1 else row['inter_crop_23'], axis=1)

# Replace inter_crop_24 based on the conditions
df['inter_crop_24'] = df.apply(lambda row: row['multi_24'] if row['inter_crop_24'] == 0 and row['multi_24'] == 1 else row['inter_crop_24'], axis=1)

# Display the updated DataFrame to verify the changes
print(df[['multi_23', 'multi_24', 'inter_crop_23', 'inter_crop_24']].head())


   multi_23  multi_24  inter_crop_23  inter_crop_24
0         0         0              0              0
1         0         0              0              0
2         0         0              0              0
3         0         0              0              0
4         0         0              0              0


In [74]:


# List of columns to check for ind_seed_23
ind_hyb_columns = [f'f_ind_hyb_{i}' for i in range(1, 7)]

# Check and ensure columns exist before using them
for col in ind_hyb_columns:
    if col not in df.columns:
        df[col] = 0  # or pd.NA or any appropriate default value

# Generate ind_seed_23 column based on the conditions
df['ind_seed_23'] = (
    (df['f_ind_hyb_1'] == 2) |
    (df['f_ind_hyb_2'] == 2) |
    (df['f_ind_hyb_3'] == 2) |
    (df['f_ind_hyb_4'] == 2) |
    (df['f_ind_hyb_5'] == 2) |
    (df['f_ind_hyb_6'] == 2)
).astype(int)

# Renaming columns to match Python conventions for readability
df.rename(columns={
    'inter_crop_23': 'Intercropping_23',
    'multi_23': 'Multistory_23',
    'inter_crop_24': 'Intercropping_24',
    'multi_24': 'Multistory_24'
}, inplace=True)

# Print the updated DataFrame to verify the changes
df[['ind_seed_23', 'Intercropping_23', 'Multistory_23', 'Intercropping_24', 'Multistory_24']].head()



,ind_seed_23,Intercropping_23,Multistory_23,Intercropping_24,Multistory_24
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [75]:
# Saving the data in csv

df.to_csv("HH_lca_FINAL_Contsruction.csv", index = False)